In [1]:
# Подключение пакетов
import os, sys, time
import uproot3 as uproot
import numpy as np
import pandas as pd
from math import floor
from mpl_toolkits import mplot3d
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from numbers import Integral
from scipy.optimize import curve_fit
from scipy.stats import norm, truncnorm, foldnorm
from itertools import compress
from pynverse import inversefunc
import warnings
from time import perf_counter


warnings.filterwarnings("ignore")

plt.rcParams['font.size'] = 16

print('Uproot version:',uproot.version.version)
print('Numpy version:', np.version.version)
print('Pandas version:', pd.__version__)
plt.ioff()

# инициализация генератора псевдослучайных чисел
# pandas version was 1.5.1

rng = np.random.default_rng(12345)

Uproot version: 3.14.4
Numpy version: 1.23.4
Pandas version: 2.0.0


In [2]:
tmp = pd.read_csv('test.csv')
tmp

,subentry,x_c,y_c,z_c,t_c,signal,m_id_event,m_id_primary,m_hits.m_photon_id_pmt,m_hits.m_photon_id_chip,...,eps_beta_4_rsigms_1_tsigms,beta_from_calc_r_4_rsigms_2_tsigms,delta_beta_4_rsigms_2_tsigms,eps_beta_4_rsigms_2_tsigms,beta_from_calc_r_4_rsigms_3_tsigms,delta_beta_4_rsigms_3_tsigms,eps_beta_4_rsigms_3_tsigms,beta_from_calc_r_4_rsigms_4_tsigms,delta_beta_4_rsigms_4_tsigms,eps_beta_4_rsigms_4_tsigms
0,7,49.92,-85.12,201.05,5.213469,True,1445.0,-211.0,492.0,56.0,...,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747
1,8,36.48,-8.80,201.05,5.029718,True,1445.0,-211.0,495.0,22.0,...,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747
2,12,126.24,-77.60,201.05,5.138784,True,1445.0,-211.0,583.0,34.0,...,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747
3,13,116.16,-67.52,201.05,5.115409,True,1445.0,-211.0,583.0,13.0,...,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747
4,14,119.52,-60.80,201.05,5.115340,True,1445.0,-211.0,583.0,23.0,...,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747
5,15,116.16,-36.48,201.05,5.090074,True,1445.0,-211.0,584.0,14.0,...,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747
6,16,116.16,-49.92,201.05,5.099505,True,1445.0,-211.0,584.0,10.0,...,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747
7,17,112.80,-33.12,201.05,5.086668,True,1445.0,-211.0,584.0,7.0,...,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747
8,18,126.24,-12.16,201.05,5.102201,True,1445.0,-211.0,585.0,37.0,...,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747
9,19,122.88,-22.24,201.05,5.098356,True,1445.0,-211.0,585.0,26.0,...,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747,1.00166,0.003312,0.331747


In [3]:
datadir = 'data'
picsdir = 'pics'

In [4]:
# отображение ключей в файле uproot в виде иерархии
def show_uproot_tree(obj, maxkeylen=12, sep='/', indent=0) -> None:
  width = maxkeylen+len(sep)
  startline = False
  if isinstance(obj, uproot.rootio.ROOTDirectory):
    print('TFile: '+obj.name.decode('utf-8'))
    startline = True
    indent = 2
  elif issubclass(type(obj), uproot.tree.TTreeMethods):
    print('TTree: '+obj.name.decode('utf-8'))
    startline = True
    indent = 4
  else:
    if len(obj.keys()) > 0:
      indent += width
      s = obj.name.decode('utf-8')[:maxkeylen]
      print(s + ' '*(maxkeylen-len(s)) + sep, end='')
    else:
      print(obj.name.decode('utf-8'))

  if len(obj.keys()) > 0:
    for i, key in enumerate(obj.keys()):
      if i>0 or startline:
        print(' '*indent, end='')
      show_uproot_tree(obj[key], indent=indent)
    indent -= width

In [5]:
filepath = os.path.join(datadir, 'farichsim_1200kevt.root')
show_uproot_tree(uproot.open(filepath))

TFile: ./farichsim_pi-pi-_45-360deg_1200.0k_ideal_2020-12-24_rndm.root
  TTree: info_sim
    info_gen    /m_num_events
                 m_z_dis
    info_rad    /m_layers    /m_layers.first
                              m_layers.second
    info_pmt    /m_name
                 m_num_side_x
                 m_num_side_y
                 m_gap
                 m_size
                 m_chip_num_size
                 m_chip_pitch
                 m_chip_size
                 m_chip_offset
                 m_focal_length
                 m_trg_window
                 m_origin_pos/m_origin_pos._2
                              m_origin_pos._1
                              m_origin_pos._0
  TTree: raw_data
    event       /m_id_event
                 m_id_primary
                 m_pos_primar/m_pos_primary._2
                              m_pos_primary._1
                              m_pos_primary._0
                 m_dir_primar/m_dir_primary._2
                              m_dir_primary._1


In [6]:
def readInfoFromRoot(filepath, verbose: bool = False) -> pd.DataFrame:
  '''
  Получение информации о моделировании из ROOT-файла в виде датафрейм формой (1, N), где N - число параметров.
  '''
  # Названия используемых колонок данных для переименования и сохранения в data frame
  idf_rename_map = {'m_num_events': 'nevents',  # число событий моделирования
                    'm_z_dis': 'zdis',  # расстояние от места рождения частицы до входа в радиатор в мм
                    'm_layers': 'nlayers',  # число слоев радиатора
                    'm_size': 'array_size',  # размер матрицы КФУ в мм
                    'm_gap': 'array_gap',  # зазор между матрицами КФУ в мм
                    'm_chip_size': 'pixel_size',  # размер пикселя КФУ в мм
                    'm_chip_pitch': 'pixel_gap',  # зазор между пикселями КФУ в мм
                    'm_chip_num_size': 'pixel_numx',  # размер матрицы КФУ в пикселях
                    'm_num_side_x': 'nxarrays', 'm_num_side_y': 'nyarrays',  # размер фотодетектора в матрицах КФУ по X и Y
                    'm_focal_length': 'distance',  # расстояние от входа в радиатор до входа в фотодетектор
                    'm_trg_window': 'trg_window_ns',  # размер временного окна в нс
                    'W': 'W',  # толщина радиатора в мм (вычисляемая)
                    'n_mean': 'n_mean',  # средний показатель преломления радиатора (вычисляемый)
                    'n_max': 'n_max',  # максимальный показатель преломления радиатора (вычисляемый)
                   }

  # Открытие ROOT-файла с данными используя Uproot https://github.com/scikit-hep/uproot3
  with uproot.open(filepath) as file:
    idf = file[b'info_sim'].pandas.df('*', flatten=False)

  # Переименование параметров
  idf.rename(columns=idf_rename_map, inplace=True, errors='ignore')

  # Получение параметров (многослойного) радиатора одинаковых для всех файлов
  n_l = idf.at[0,'m_layers.first']  # показатели преломления слоёв
  w_l = idf.at[0,'m_layers.second']  # толщины слоёв радиатора

  W = w_l.sum()  # суммарная толщина всех слоёв
  n_mean = n_l.mean()  # средний показатель преломления
  n_max = n_l.max()  # максимальный показатель преломления

  # Добавление вычисляемых параметров в idf
  idf['W'] = W
  idf['n_mean'] = n_mean
  idf['n_max'] = n_max

  # Сохранение нужных параметров
  idf = idf[idf_rename_map.values()]

  if verbose:
    for name in idf.columns:
      print(f'{name}: {idf.at[0, name]}')

  return idf

In [7]:
idf = readInfoFromRoot(filepath)

In [8]:
idf

,nevents,zdis,nlayers,array_size,array_gap,pixel_size,pixel_gap,pixel_numx,nxarrays,nyarrays,distance,trg_window_ns,W,n_mean,n_max
entry,,,,,,,,,,,,,,,
0,1200000,1.0,4,26.68,1.0,3.16,0.2,8,30,30,200.0,20.0,35.0,1.0454,1.05


In [9]:
def genChunkFromRoot(filepath, eventchunksize=2000, noisefreqpersqmm: float = 2e6, noiseTimeRange: float = (0, 7), shiftSignalTimes: bool = True,
                     edfstore: pd.HDFStore = None, verbose: bool = True) -> pd.DataFrame:
  """
    Генератор событий из ROOT-файла в виде датафрейма. Число событий eventchunksize, читаемых генератором за один раз, должен выбираться так,
    чтобы все данные с учетом добавляемых шумовых срабатываний умещались в размер ОЗУ.

    Параметры:
    filepath - путь к ROOT-файлу для чтения.
    eventchunksize - число событий, загружаемых из ROOT-файла за один вызов.
    noisefreqpersqmm - частота темновых срабатываний на единицу активной площади фотодетектора в с^{-1}*мм^{-2}, подмешиваемых к событиям;
                       максимальное значение параметра, которое имеет смысл рассматривать, 2e6.
    noiseTimeRange - (start, stop) - tuple, задающий временной интервал генерации шума в наносекундах.
    edfstore - HDF-хранилище для записи датафрейма "edf"; данные добавляются к уже записанным в хранилище.
    verbose - флаг отладочной печати.

    Описание условий моделирования:
    Ось Z направлена по нормали к плоскости радиатора от радиатора к фотодетектору.
    Оси X и Y паралельны осям симметрии матрицы фотодетектора.
    Первичная частица (отрицательный пион) вылетает на расстоянии zdis=1 мм перед радиатором в его сторону
    Начальное положение частицы случайно разбрасывается по X и Y в квадрате со стороной (pixel_size+pixel_gap).
    Направление частицы случайно разбрасывается в телесном угле в пределах theta_p=[0, π/4], phi_p=[0, 2π].
    Скорость частицы случайно и равномерно разбрасывается от 0.957 до 0.999 скорости света.
  """
  global rng

  # Данные о частице (для переименования и сохранения)
  part_rename_map = {'m_hits': 'nhits',                # число срабатываний в событии
                     'm_pos_primary._0': 'x_p',        # X-координата вылета частицы в мм
                     'm_pos_primary._1': 'y_p',        # Y-координата вылета частицы в мм
                     'm_pos_primary._2': 'z_p',        # Z-координата вылета частицы в мм
                     'm_dir_primary._0': 'nx_p',       # X-компонента единичного вектора направления частицы
                     'm_dir_primary._1': 'ny_p',       # Y-компонента единичного вектора направления частицы
                     'm_dir_primary._2': 'nz_p',       # Z-компонента единичного вектора направления частицы
                     'm_beta_primary': 'beta',         # скорость частицы в единицах скорости света
                     'm_theta_primary': 'theta_p',     # полярный угол направления частицы в радианах
                     'm_phi_primary': 'phi_p',         # азимутальный угол направления частицы в радианах
                     'm_momentum_primary': 'momentum'  # импульс частицы в МэВ/c
                    }
  
  # Наблюдаемые данные о срабатываниях (для переименования и сохранения)
  hit_rename_map = {'m_hits.m_photon_pos_chip._0': 'x_c',  # X-координата срабатывания в мм
                    'm_hits.m_photon_pos_chip._1': 'y_c',  # Y-координата срабатывания в мм
                    'm_hits.m_photon_pos_chip._2': 'z_c',  # Z-координата срабатывания в мм
                    'm_hits.m_photon_time': 't_c'          # время срабатывания в нс
                   }

  other_rename_map = {'m_id_event': '',                         #
                      'm_id_primary': '',                       #
                      'm_hits.m_photon_id_pmt': '',             #
                      'm_hits.m_photon_id_chip': '',            #
                      'm_hits.m_photon_id_layer': '',           #
                      'm_hits.m_photon_id_track': '',           #
                      'm_hits.m_photon_id_track_parent': '',    #
                      'm_hits.m_photon_id_hit': '',             #
                      'm_hits.m_photon_wl': '',                 #
                      'm_hits.m_photon_pos_exact._0': '',       #
                      'm_hits.m_photon_pos_exact._1': '',       #
                      'm_hits.m_photon_pos_exact._2': '',       #
                      'm_hits.m_photon_pos_vertex._0': '',      #
                      'm_hits.m_photon_pos_vertex._1': '',      #
                      'm_hits.m_photon_pos_vertex._2': '',      #
                      'm_hits.m_photon_dir_vertex._0': '',      #
                      'm_hits.m_photon_dir_vertex._1': '',      #
                      'm_hits.m_photon_dir_vertex._2': '',      #
                      'm_hits.m_hit_is_fittable': '',           #
                      'm_hits.m_photon_theta.first': '',        #
                      'm_hits.m_photon_theta.second': '',       #
                      'm_hits.m_photon_phi.first': '',          #
                      'm_hits.m_photon_phi.second': '',         #
                      }

  for key in other_rename_map:
    other_rename_map.update({key: key})
  # Наименования колонок для сохранения в датафрейм
  edfcolstosave = list(part_rename_map.values()) + list(hit_rename_map.values())

  # Чтение параметров моделирования
  idf = readInfoFromRoot(filepath)

  # Определения параметров фотодетектора для генерации темнового шума
  pixel_size, pixel_gap = idf.at[0, 'pixel_size'], idf.at[0, 'pixel_gap']
  array_size, array_gap = idf.at[0, 'array_size'], idf.at[0, 'array_gap']
  nxpixels_arr = idf.at[0, 'pixel_numx']
  nxpixels_tot = idf.at[0, 'nxarrays']*nxpixels_arr
  igrid = np.arange(nxpixels_tot//2)
  xpnts = array_gap/2 + (igrid//nxpixels_arr)*(array_size+array_gap) + (igrid%nxpixels_arr)*(pixel_size+pixel_gap) + pixel_size/2
  xpnts = np.sort(np.append(-xpnts, xpnts)).astype('float32')
  xgrid, ygrid = np.meshgrid(xpnts, xpnts)
  xgrid = xgrid.reshape(xgrid.size)
  ygrid = ygrid.reshape(ygrid.size)
 
  def addNoise(partdf: pd.DataFrame, hitdf: pd.DataFrame) -> pd.DataFrame:
    ''' 
    Генерация темновых срабатываний темнового шума и добавление в датафрейм (без учета "мёртвого" времени пикселя).
    partdf - датафрейм для частиц
    hitdf - датафрейм для срабатываний
    '''
    assert(np.isclose(pixel_size*nxpixels_arr+pixel_gap*(nxpixels_arr-1), array_size))
    nevents = partdf.shape[0]  # число событий
    
    # среднее число шумовых срабатываний на событие
    munoise = (noiseTimeRange[1]-noiseTimeRange[0])*1e-9*noisefreqpersqmm*(pixel_size**2)*(nxpixels_tot**2)

    print(f'    Generate noise with DCR per mm^2 {noisefreqpersqmm}, mean number of hits per event: {munoise:.2f}.')

    noisehits = rng.poisson(munoise, nevents)   # генерация массива числа шумовых срабатываний в событиях по пуассоновскому распределению
    Ndc = int(noisehits.sum())                  # общее число шумовых срабатываний (скаляр)
    signalhits = partdf['nhits'].to_numpy()     # массив числа сигнальных срабатываний по событиям

    # случайное смещение сигнальных срабатываний в пределах временного окна генерации шума
    if shiftSignalTimes:
      hitdf['t_c'] += np.repeat(rng.uniform(0, noiseTimeRange[1]-2, nevents), partdf['nhits'])

    hitdf['signal'] = np.ones(signalhits.sum(), bool)  # разметка сигнальных срабатываний значением 'signal' True
    if Ndc == 0:    # если нет шумовых срабатываний
      return hitdf  # возвращаем исходный датафрейм с добавлением колонки 'signal'

    ich = rng.choice(xgrid.size, Ndc)           # генерация случайных номеров сработавших каналов с возможным повтором
    xh = xgrid[ich]                             # x-координата сработавших каналов
    yh = ygrid[ich]                             # y-координата сработавших каналов
    # zh = hitdf.at[(0, 0), 'z_c']                # z-координата срабатываний (скаляр)
    zh = 201.050003
    th = rng.uniform(noiseTimeRange[0], noiseTimeRange[1], size=Ndc) # генерация времён срабатываний по однородному распределению
   
    # нумерация шумовых срабатываний по событиям
    ievent = np.repeat(partdf.index, noisehits) # массив номеров событий для записи в датафрейм
    ihit = np.zeros(Ndc, 'int64')               # инициализация массива номеров срабатываний для записи в датафрейм
    index = 0
    for i in range(nevents):
      ihit[index:index+noisehits[i]] = signalhits[i] + np.arange(noisehits[i])
      index += noisehits[i]
    
    # создание датафрейма с шумовыми срабатываниями того же формата, что hitdf
    noisedf = pd.DataFrame({'x_c': xh, 'y_c': yh, 'z_c': zh, 't_c': th, 'signal': np.zeros(Ndc, bool)},
                           index=pd.MultiIndex.from_arrays((ievent, ihit), names=('entry', 'subentry')))

    # TO DO: случайное смещение кольца в фотодетекторе (сдвиг координат сигнальных хитов).
    # Сложность с реализацией для неравномерной сетки пикселей, т.к. зазоры между матрицами больше зазоров между пикселями в матрице.
    # Проще сделать в моделировании.

    # сливаем сигнальный и шумовой датафрейм и сортируем указатель событий и срабатываний
    hitdf2 = pd.concat((hitdf, noisedf), copy=False).sort_index(level=('entry', 'subentry'))

    # обновляем количества срабатываний в partdf, добавляя количества шумовых срабатываний по событиям
    partdf['nhits'] += noisehits

    return hitdf2


  nFileEvents = idf.at[0, 'nevents']
  # print(f'Processing ROOT file {filepath} with {nFileEvents} simulated events...', flush=True)
  
  # Цикл чтения кусков ROOT-файла
  for partdf, hitdf, otherdf in zip(uproot.pandas.iterate(filepath, "raw_data", part_rename_map.keys(), entrysteps=eventchunksize),
                           uproot.pandas.iterate(filepath, "raw_data", hit_rename_map.keys(), entrysteps=eventchunksize, flatten=True),
                           uproot.pandas.iterate(filepath, "raw_data", other_rename_map.keys(), entrysteps=eventchunksize, flatten=True)):
    # print('\n  Processing next chunk...')

    # Переименование колонок
    partdf.rename(columns=part_rename_map, inplace=True, errors='raise')
    hitdf.rename(columns=hit_rename_map, inplace=True, errors='raise')
    otherdf.rename(columns=other_rename_map, inplace=True, errors='raise')

    partdf = partdf.astype('float32', copy=False)
    partdf['nhits'] = partdf['nhits'].astype('int32', copy=False)
    hitdf = hitdf.astype('float32', copy=False)
    otherdf = otherdf.astype('float32', copy=False)

    # Генерация и добавление шумовых срабатываний
    hitdf = addNoise(partdf, hitdf)

    # print(f'    {hitdf.index.levels[0].size} entries with {hitdf.shape[0]} hits to process')

    # Слияние данных событий и срабатываний
    edf = hitdf.join(otherdf)
    edf = edf.join(partdf, on='entry')
    #edf = hitdf.join(partdf, on='entry')
    #edf = edf.join(otherdf, on='entry')
    if verbose:
      pass
      # print(edf)

    if edfstore is not None:
      # print(f'    Saving edf chunk...')
      edfstore.put('edf', edf, format='table', append=True)

    yield edf

In [10]:
# Чтение и домоделирование одного "куска" данных
#edf = next(genChunkFromRoot(filepath, 1450, noisefreqpersqmm=1000000))
edf = next(genChunkFromRoot(filepath, 10000, noisefreqpersqmm=0))

    Generate noise with DCR per mm^2 0, mean number of hits per event: 0.00.


In [11]:
def edf_to_bdfOld(edf_col: pd.Series, bdf: pd.DataFrame):
  to_bdf = []
  for _, sub in edf_col.groupby(level=0):
    for subsub in sub:
      to_bdf.append(subsub)
      break
  bdf[edf_col.name] = pd.Series(to_bdf)

In [12]:
def edf_to_bdf(edf_col: pd.Series, bdf: pd.DataFrame):
  to_bdf = [sub.iloc[0] for _, sub in edf_col.groupby(level=0)]
  bdf[edf_col.name] = pd.Series(to_bdf)

In [13]:
edf

x_c         y_c         z_c       t_c  signal   
entry subentry                                                         
0     0         -70.879997 -226.880005  201.050003  2.188593    True  \
      1         -53.279999 -309.920013  201.050003  2.396409    True   
      2         -36.480000 -306.559998  201.050003  2.382249    True   
      3         -49.919998 -184.960007  201.050003  2.073419    True   
      4         -12.160000 -288.959991  201.050003  2.326930    True   
...                    ...         ...         ...       ...     ...   
9999  40        178.240005  -80.959999  201.050003  3.695349    True   
      41        178.240005  -74.239998  201.050003  3.687293    True   
      42        226.880005  -53.279999  201.050003  3.892318    True   
      43        274.720001 -209.279999  201.050003  4.102964    True   
      44        251.199997  -18.879999  201.050003  3.840951    True   

                m_id_event  m_id_primary  m_hits.m_photon_id_pmt   
entry subentry                                                     
0     0                1.0        -211.0                   367.0  \
      1                1.0        -211.0                   394.0   
      2                1.0        -211.0                   394.0   
      3                1.0        -211.0                   399.0   
      4                1.0        -211.0                   425.0   
...                    ...           ...                     ...   
9999  40           10000.0        -211.0                   643.0   
      41           10000.0        -211.0                   643.0   
      42           10000.0        -211.0                   704.0   
      43           10000.0        -211.0                   728.0   
      44           10000.0        -211.0                   735.0   

                m_hits.m_photon_id_chip  m_hits.m_photon_id_layer  ...   
entry subentry                                                     ...   
0     0                            31.0                       4.0  ...  \
      1                             7.0                       4.0  ...   
      2                            48.0                       4.0  ...   
      3                            11.0                       2.0  ...   
      4                            37.0                       3.0  ...   
...                                 ...                       ...  ...   
9999  40                           25.0                       1.0  ...   
      41                           27.0                       3.0  ...   
      42                            9.0                       4.0  ...   
      43                           60.0                       4.0  ...   
      44                            3.0                       4.0  ...   

                     x_p       y_p  z_p      nx_p      ny_p      nz_p   
entry subentry                                                          
0     0         1.275724  1.576625  0.0 -0.073019 -0.695325  0.714977  \
      1         1.275724  1.576625  0.0 -0.073019 -0.695325  0.714977   
      2         1.275724  1.576625  0.0 -0.073019 -0.695325  0.714977   
      3         1.275724  1.576625  0.0 -0.073019 -0.695325  0.714977   
      4         1.275724  1.576625  0.0 -0.073019 -0.695325  0.714977   
...                  ...       ...  ...       ...       ...       ...   
9999  40        1.499871 -1.121928  0.0  0.419547 -0.313757  0.851784   
      41        1.499871 -1.121928  0.0  0.419547 -0.313757  0.851784   
      42        1.499871 -1.121928  0.0  0.419547 -0.313757  0.851784   
      43        1.499871 -1.121928  0.0  0.419547 -0.313757  0.851784   
      44        1.499871 -1.121928  0.0  0.419547 -0.313757  0.851784   

                    beta   theta_p     phi_p    momentum  
entry subentry                                            
0     0         0.966818  0.774205  4.607758  528.209290  
      1         0.966818  0.774205  4.607758  528.209290  
      2         0.966818  0.774205  4.607758  528.20

In [14]:
def recoAngles(edf: pd.DataFrame, idf: pd.DataFrame, rotation_mode = False):
  '''
  Геометрическая реконструкция углов фотонов относительно направления частицы.
  Из координат срабатываний и частиц вычисляются углы theta_c, phi_c и время вылета фотонов t_c_orig и добавляются к edf.
  '''
  r0 = edf.loc[:, ('x_p', 'y_p', 'z_p')].to_numpy()
  if rotation_mode:
    r = edf.loc[:, ('rotated_x', 'rotated_y', 'rotated_z')].to_numpy()
    # n0 = edf.loc[:, ('rotated_nx_p', 'rotated_ny_p', 'rotated_nz_p')].to_numpy()
    n0 = edf.loc[:, ('recalculated_nx_p', 'recalculated_ny_p', 'recalculated_nz_p')].to_numpy()
  else:
    r  = edf.loc[:, ('x_c', 'y_c', 'z_c')].to_numpy()
    n0 = edf.loc[:, ('nx_p', 'ny_p', 'nz_p')].to_numpy()

  speedOfLight_mmperns = 299.792458 # мм/нс

  # расстояние от радиатора до детектора
  dist = float(idf['distance'])

  # толщина радиатора
  W = float(idf['W'])

  # расстояние от точки вылета частицы до входной плоскости радиатора
  rad_pos = float(idf['zdis'])

  # полное число срабатываний
  N = edf.shape[0]

  # координаты точки пересечения трека с ФД
  if not rotation_mode:
    y_i = r0[:,1] + (dist + rad_pos) * n0[:,1] / n0[:,2] # r0[:,1] + (dist + W + rad_pos) * n0[:,1] / n0[:,2]   #   r0[:,1] + (dist + rad_pos) * n0[:,1] / n0[:,2]
    x_i = r0[:,0] + (y_i - r0[:,1]) * n0[:,0] / n0[:,1] # r0[:,0] + (y_i - r0[:,1]) * n0[:,0] / n0[:,1]    #     r0[:,0] + (dist + rad_pos) * n0[:,0] / n0[:,2]
    edf['x_i'] = x_i
    edf['y_i'] = y_i
    edf['r_p_c'] = np.sqrt((r0[:,0] - x_i) ** 2 + (r0[:,1] - y_i) ** 2 + (r0[:,2] - r[:,2]) ** 2)
    edf['r_c'] = np.sqrt((x_i - edf['x_c']) ** 2 + (y_i - edf['y_c']) ** 2)

  if rotation_mode:
    n_mean = float(idf['n_mean'])

    edf['rotated_r_c'] = np.sqrt((edf['rotated_x_i'] - edf['rotated_x']) ** 2 + (edf['rotated_y_i'] - edf['rotated_y']) ** 2)

    rotated_r_c = edf['rotated_r_c'].to_numpy()
    # r_p_c = edf['r_p_c'].to_numpy()
    beta = edf['beta'].to_numpy()
    r_p_c = dist # or + W/2 ???

    edf['beta_from_true_r'] = np.sqrt(rotated_r_c ** 2 + r_p_c ** 2) / (n_mean * r_p_c)
    edf['true_r_from_beta'] = r_p_c * np.sqrt((n_mean * beta) ** 2 - 1)

    avg_betas = []
    for _, subentry in edf['beta_from_true_r'].groupby(level=0):
      avg_beta = subentry.mean()
      for __ in subentry:
        avg_betas.append(avg_beta)
    edf['beta_from_true_r_mean'] = avg_betas
  # косинусы и синусы сферических углов направления частицы
  costheta, sintheta = n0[:,2], np.sqrt(n0[:,0]**2+n0[:,1]**2)
  phi = np.arctan2(n0[:,1], n0[:,0])
  cosphi, sinphi = np.cos(phi), np.sin(phi)

  # номинальная точка вылета фотонов
  ro = r0 + (W/2+rad_pos)/n0[:,2].reshape(N,1)*n0

  """
  Преобразование в СК частицы
  𝑢𝑥 = cos 𝜃(𝑣𝑥 cos 𝜙 + 𝑣𝑦 sin 𝜙) − 𝑣𝑧 sin 𝜃,
  𝑢𝑦 = −𝑣𝑥 sin 𝜙 + 𝑣𝑦 cos 𝜙,
  𝑢𝑧 = sin 𝜃(𝑣𝑥 cos 𝜙 + 𝑣𝑦 sin 𝜙) + 𝑣𝑧 cos 𝜃.
  """

  # вектор направления фотона в лабораторной СК
  s = (r-ro)
  snorm = np.linalg.norm(s, axis=1, keepdims=True)
  v = s / snorm
  if not rotation_mode:
    edf['t_c_orig'] = edf['t_c'] - (snorm / speedOfLight_mmperns).reshape(N)

  # освобождение памяти при необходимости
  #del r0, n0, ro, r, s

  U = np.stack((np.stack((costheta*cosphi, costheta*sinphi, -sintheta)),
                np.stack((-sinphi,         cosphi,          np.full(N, 0.))),
                np.stack((sintheta*cosphi, sintheta*sinphi, costheta)))).transpose(2,0,1)

  # единичный вектор направления фотона в СК частицы
  u = (U @ v.reshape(N,3,1)).reshape(N,3)

  # сферические углы фотона в СК частицы
  if rotation_mode:
    edf['rotated_theta_c'] = np.arccos(u[:,2])
    edf['rotated_phi_c'] = np.arctan2(-u[:,1], -u[:,0])
  else:
    edf['theta_c'] = np.arccos(u[:,2])
    edf['phi_c'] = np.arctan2(-u[:,1], -u[:,0])
    avg_thetas = []
    for _, subentry in edf['theta_c'].groupby(level=0):
      avg_theta = subentry.mean()
      for __ in subentry:
        avg_thetas.append(avg_theta)
    edf['theta_c_mean'] = avg_thetas

In [15]:
def applySpaceCut(edf: pd.DataFrame) -> pd.DataFrame:
  return edf[(abs(edf['x_c'] - edf['x_i']) <= 220) & (abs(edf['y_c'] - edf['y_i']) <= 220)]

In [16]:
def signalLength(edf: pd.DataFrame):
  t_signal = edf[edf['signal'] ==  True]['t_c']
  t = []
  for _, subentry in t_signal.groupby(level=0):
    t_min = np.min(subentry)
    t_max = np.max(subentry)
    t.extend(t_max - t_min for __ in subentry)
  t = np.array(t)
  edf['signal_length'] = t

In [17]:
def rotateVectorAroundZaxis(r0, az, eps=1e-6) -> np.array:
  x = r0[0] * np.cos(az) - r0[1] * np.sin(az)
  y = r0[0] * np.sin(az) + r0[1] * np.cos(az)
  z = r0[2]
  r = np.array([x, y ,z])
  return r

In [18]:
def testRotation2(edf: pd.DataFrame):
  R_p = edf.loc[:, ('x_p', 'y_p', 'z_p')].to_numpy()
  R = edf.loc[:, ('x_c', 'y_c', 'z_c')].to_numpy()
  R_i = edf.loc[:, ('x_i', 'y_i', 'z_c')].to_numpy()
  N = edf.loc[:, ('nx_p', 'ny_p', 'nz_p')].to_numpy()
  R_i_new = []
  R_new = []

  for r_i, ir in zip(R_i, R):

    rR = np.sqrt(r_i[0] ** 2 + r_i[1] ** 2 + r_i[2] ** 2)
    tR = np.arccos(r_i[2] / rR)
    pR = np.arctan2(r_i[2], r_i[1])
    rC = [0, 0, r_i[2]]

    # rotate back? (el->circle)
    rt0 = rotateVectorAroundZaxis(r_i, -pR)
    rt1 = np.array([rt0[2] * np.tan(np.arctan2(rt0[0], rt0[2]) - tR), rt0[1], rt0[2]])
    rCf = rotateVectorAroundZaxis(rt1, pR)
    rCf = rt1
    rCfabs = np.sqrt(rCf[0] ** 2 + rCf[1] ** 2 + rCf[2] ** 2)
    tCf = np.arccos(rCf[2] / rCfabs)
    pCf = np.arctan2(rCf[1], rCf[1])
    R_i_new.append(rCf)

    irt0 = rotateVectorAroundZaxis(ir, -pR)
    irt1 = np.array([irt0[2] * np.tan(np.arctan2(irt0[0], irt0[2]) - tR), irt0[1], irt0[2]])
    irf = rotateVectorAroundZaxis(irt1, pR)
    irf = irt1
    irfabs = np.sqrt(irf[0] ** 2 + irf[1] ** 2 + irf[2] ** 2)
    itf = np.arccos(irf[2] / irfabs)
    ipf = np.arctan2(irf[1], irf[0])
    R_new.append(irf)

  R_i_new = np.array(R_i_new)
  R_new = np.array(R_new)

  edf['rotated_x'] = R_new[:,0]
  edf['rotated_y'] = R_new[:,1]
  edf['rotated_z'] = R_new[:,2]

  edf['rotated_x_i'] = R_i_new[:,0]
  edf['rotated_y_i'] = R_i_new[:,1]
  edf['rotated_z_i'] = R_i_new[:,2]

  edf['rotated_nx_p'] = (edf['rotated_x_i'] - edf['x_p']).to_numpy()
  edf['rotated_ny_p'] = (edf['rotated_y_i'] - edf['y_p']).to_numpy()
  edf['rotated_nz_p'] = (edf['rotated_z_i'] - edf['z_p']).to_numpy()

  rotated_n = edf.loc[:, ('rotated_nx_p', 'rotated_ny_p', 'rotated_nz_p')].to_numpy()
  rotated_n = rotated_n / np.linalg.norm(rotated_n, axis=1, keepdims=True)

  edf['rotated_nx_p'] = rotated_n[:,0]
  edf['rotated_ny_p'] = rotated_n[:,1]
  edf['rotated_nz_p'] = rotated_n[:,2]

In [19]:
def planeRecalculationOld(edf: pd.DataFrame, idf: pd.DataFrame):
  R_p = edf.loc[:, ('x_p', 'y_p', 'z_p')].to_numpy()
  R = edf.loc[:, ('x_c', 'y_c', 'z_c')].to_numpy()
  R_i = edf.loc[:, ('x_i', 'y_i', 'z_c')].to_numpy()
  N = edf.loc[:, ('nx_p', 'ny_p', 'nz_p')].to_numpy()
  R_new = []
  for r_p, r, r_i, n in zip(R_p, R, R_i, N):
    alpha = (idf.W / 2 + idf.zdis - r_p[2]) / n[2] # attempt to fix wrong photon starting point
    # r_d = r_p
    r_d = r_p + n * float(alpha) # r_p -> r_d
    u = r - r_d
    dot = n @ u
    w = r_d - r_i
    fac = - n @ w / dot
    u = u * fac
    R_new.append(r_d + u)
  R_new = np.array(R_new)
  edf['recalculated_x'] = R_new[:,0]
  edf['recalculated_y'] = R_new[:,1]
  edf['recalculated_z'] = R_new[:,2]

In [20]:
def planeRecalculation(edf: pd.DataFrame, idf: pd.DataFrame):
    R_p = edf[['x_p', 'y_p', 'z_p']].to_numpy()
    R = edf[['x_c', 'y_c', 'z_c']].to_numpy()
    R_i = edf[['x_i', 'y_i', 'z_c']].to_numpy()
    N = edf[['nx_p', 'ny_p', 'nz_p']].to_numpy()
    dist = idf.W / 2 + idf.zdis
    alpha = (float(dist) - R_p[:,2]) / N[:,2]
    r_d = R_p + N * alpha[:, np.newaxis]

    u = R - r_d
    dot = np.sum(N * u, axis=1)
    w = r_d - R_i
    fac = -np.sum(N * w, axis=1) / dot
    u *= fac[:, np.newaxis]

    R_new = r_d + u



    edf['recalculated_x'] = R_new[:,0]
    edf['recalculated_y'] = R_new[:,1]
    edf['recalculated_z'] = R_new[:,2]

In [21]:
def primaryDirectionRecalculation(edf: pd.DataFrame):
  N = edf.loc[:, ('nx_p', 'ny_p', 'nz_p')].to_numpy()
  M = []
  for n in N:
    # C = np.stack((np.array([-(n[1] ** 2 + n[2] ** 2) / n[0], 0 , n[0]]),
    #               np.array([n[1], - n[2], n[1]]),
    #               np.array([n[2], n[1], n[2]])))

    # C_inv = np.array([np.array([- n[0] / (n[0] ** 2 + n[1] ** 2 + n[2] ** 2), n[0] ** 2 * n[1] / (n[1] ** 4 + n[0] ** 2 * n[1] ** 2 + n[2] ** 4 + n[0] ** 2 * n[2] ** 2 + 2 * n[1] ** 2 * n[2] ** 2) , n[0] ** 2 * n[2] / (n[1] ** 4 + n[0] ** 2 * n[1] ** 2 + n[2] ** 4 + n[0] ** 2 * n[2] ** 2 + 2 * n[1] ** 2 * n[2] ** 2)]),
    #               np.array([0, - n[2] / (n[1] ** 2 + n[2] ** 2), n[1] / (n[1] ** 2 + n[2] ** 2)]),
    #               np.array([n[0] / (n[0] ** 2 + n[1] ** 2 + n[2] ** 2), n[1] / (n[0] ** 2 + n[1] ** 2 + n[2] ** 2), n[2] / (n[0] ** 2 + n[1] ** 2 + n[2] ** 2)])])
    M.append([0, 0, 1])
    # print(n)
    # print(C_inv)
    # print(C_inv @ n)
    # break
  M = np.array(M)
  edf['recalculated_nx_p'] = M[:,0]
  edf['recalculated_ny_p'] = M[:,1]
  edf['recalculated_nz_p'] = M[:,2]

In [22]:
def planeRotationOld(edf: pd.DataFrame):   # Deprecated
  R = edf.loc[:, ('recalculated_x', 'recalculated_y', 'recalculated_z')].to_numpy()
  R_i = edf.loc[:, ('x_i', 'y_i', 'z_c')].to_numpy()
  N = edf.loc[:, ('nx_p', 'ny_p', 'nz_p')].to_numpy() # N
  M = np.array([[0, 0, 1]])                           # M
  M = M[0]
  new_r = []
  new_r_c = []
  i = 0
  for n, r in zip(N, R):
    c = np.dot(n, M) / (np.linalg.norm(M) * np.linalg.norm(n))

    axis = np.cross(n, M) / np.linalg.norm(np.cross(n, M))
    x = axis[0]
    y = axis[1]
    z = axis[2]
    s = np.sqrt(1-c*c)
    C = 1-c
    rmat = np.array([[x*x*C+c, x*y*C-z*s, x*z*C+y*s],
                     [y*x*C+z*s, y*y*C+c, y*z*C-x*s],
                     [z*x*C-y*s, z*y*C+x*s, z*z*C+c]])
    rmat = np.reshape(rmat, (3, 3))
    coord = np.dot(rmat, r)
    #coord[2] = 201.050003
    if abs(coord[0]) >= 500 or abs(coord[1]) >= 500:
      coord[0] = 5000
      coord[1] = 5000
    new_r.append(coord)

  for n, r in zip(N, R_i):
    c = np.dot(n, M) / (np.linalg.norm(M) * np.linalg.norm(n))

    axis = np.cross(n, M) / np.linalg.norm(np.cross(n, M))
    x = axis[0]
    y = axis[1]
    z = axis[2]
    s = np.sqrt(1-c*c)
    C = 1-c
    rmat = np.array([[x*x*C+c, x*y*C-z*s, x*z*C+y*s],
                     [y*x*C+z*s, y*y*C+c, y*z*C-x*s],
                     [z*x*C-y*s, z*y*C+x*s, z*z*C+c]])
    rmat = np.reshape(rmat, (3, 3))
    coord = np.dot(rmat, r)
    #coord[2] = 201.050003
    if abs(coord[0]) >= 500 or abs(coord[1]) >= 500:
      coord[0] = 5000
      coord[1] = 5000
    new_r_c.append(coord)


  new_r = np.array(new_r)
  new_r_c = np.array(new_r_c)


  #print(new_r)
  edf['rotated_x'] = new_r[:,0]
  edf['rotated_y'] = new_r[:,1]
  edf['rotated_z'] = new_r[:,2]

  edf['rotated_x_i'] = new_r_c[:,0]
  edf['rotated_y_i'] = new_r_c[:,1]
  edf['rotated_z_i'] = new_r_c[:,2]

  edf['rotated_nx_p'] = (edf['rotated_x_i'] - edf['x_p']).to_numpy()
  edf['rotated_ny_p'] = (edf['rotated_y_i'] - edf['y_p']).to_numpy()
  edf['rotated_nz_p'] = (edf['rotated_z_i'] - edf['z_p']).to_numpy()

  rotated_n = edf.loc[:, ('rotated_nx_p', 'rotated_ny_p', 'rotated_nz_p')].to_numpy()
  rotated_n = rotated_n / np.linalg.norm(rotated_n, axis=1, keepdims=True)

  edf['rotated_nx_p'] = rotated_n[:,0]
  edf['rotated_ny_p'] = rotated_n[:,1]
  edf['rotated_nz_p'] = rotated_n[:,2]

In [23]:
def planeRotation(edf: pd.DataFrame):
  R = edf[['recalculated_x', 'recalculated_y', 'recalculated_z']].to_numpy()
  R_i = edf[['x_i', 'y_i', 'z_c']].to_numpy()
  N = edf[['nx_p', 'ny_p', 'nz_p']].to_numpy() # N
  M = np.array([0, 0, 1])                           # M
  c = np.dot(N, M) / (np.linalg.norm(M) * np.linalg.norm(N, axis=1))
  axis = np.cross(N, np.broadcast_to(M, (N.shape[0], 3))) / np.linalg.norm(np.cross(N, np.broadcast_to(M, (N.shape[0], 3))), axis=1, keepdims=True)
  x, y, z = axis.T
  s = np.sqrt(1-c*c)
  C = 1-c
  rmat = np.array([
      [x*x*C+c, x*y*C-z*s, x*z*C+y*s],
      [y*x*C+z*s, y*y*C+c, y*z*C-x*s],
      [z*x*C-y*s, z*y*C+x*s, z*z*C+c]])
  # print(rmat.shape)
  # print(R.shape)
  # print(rmat[:, :, 0])
  # print(R[0])
  # print(rmat[:, :, 0] @ R[0])
  rotated_R = np.matmul(rmat.transpose((2, 0, 1)), R[:, :, np.newaxis])
  rotated_R = np.squeeze(rotated_R, axis=-1).transpose().T
  rotated_R_i = np.matmul(rmat.transpose((2, 0, 1)), R_i[:, :, np.newaxis])
  rotated_R_i = np.squeeze(rotated_R_i, axis=-1).transpose().T
  # print(rotated_R[0])
  maskR = np.logical_or(abs(rotated_R[:, 0]) >= 500, abs(rotated_R[:, 1]) >= 500)
  maskR_i = np.logical_or(abs(rotated_R_i[:, 0]) >= 500, abs(rotated_R_i[:, 1]) >= 500)
  rotated_R[maskR] = [5000, 5000, 0]
  rotated_R_i[maskR_i] = [5000, 5000, 0]
  rotated_n = (rotated_R_i - edf[['x_p', 'y_p', 'z_p']].to_numpy()) / np.linalg.norm(rotated_R_i - edf[['x_p', 'y_p', 'z_p']].to_numpy(), axis=1, keepdims=True)
  edf['rotated_x'] = rotated_R[:,0]
  edf['rotated_y'] = rotated_R[:,1]
  edf['rotated_z'] = rotated_R[:,2]
  edf['rotated_x_i'] = rotated_R_i[:,0]
  edf['rotated_y_i'] = rotated_R_i[:,1]
  edf['rotated_z_i'] = rotated_R_i[:,2]
  edf['rotated_nx_p'] = rotated_n[:,0]
  edf['rotated_ny_p'] = rotated_n[:,1]
  edf['rotated_nz_p'] = rotated_n[:,2]

In [24]:
def applySecondSpaceCut(edf: pd.DataFrame) -> pd.DataFrame:
  return edf[(abs(edf['rotated_x'] - edf['rotated_x_i']) <= 220) & (abs(edf['rotated_y'] - edf['rotated_y_i']) <= 220)]

In [25]:
def optimalWidth(edf: pd.DataFrame):
  def func_to_fit(x, c, loc, scale):
    return foldnorm.pdf(x, c=c, loc=loc, scale=scale)
  r_r_c = edf['rotated_r_c'].to_numpy()
  all_widths = []
  for i, subentry in edf['rotated_r_c'].groupby(level=0):
    # data = subentry[(edf['rotated_r_c'] < subentry[0].mean() + 20) and (edf['rotated_r_c'] > subentry[0].mean() - 20)]
    data = subentry[edf['rotated_r_c'] < 80]
    std = np.std(data)
    if std > 0:
      all_widths.append(std)

  # print(np.isfinite(all_widths))
  a = list(compress(all_widths, np.isfinite(all_widths)))
  print(np.mean(a))
  bin_heights, bin_borders = np.histogram(a, bins='auto', density=True)
  bin_width = np.diff(bin_borders)
  bin_centers = bin_borders[:-1] + bin_width / 2
  # print(bin_centers)
  # params, cov = curve_fit(func_to_fit, bin_centers, bin_heights, maxfev=500000)
  a = np.array(a)
  params = norm.fit(a)
  print(params)
  # plt.plot(bin_centers, norm.pdf(bin_centers, *params))
  # plt.plot(bin_centers, bin_heights)
  plt.hist(a, bins='auto', density=True)
  plt.xlabel('Width, mm')
  plt.ylabel(f'Normed Entries per {round(bin_width[0], 2)} mm')
  plt.show()

  return params[0]

In [26]:
# optimalWidth(edf)

In [27]:
def local_sum_2d(event, r_slices, t_slices, square_counts, max_index, n, m, timestep, t_window_width, method='N/r'):
  # cut_event = event[(event.t_c <= t_slices[np.clip(max_index[1] + m, a_min=0, a_max=10)]) & (event.t_c >= t_slices[np.clip(max_index[1] - m, a_min=0, a_max=10)]) &
  #                   (event.rotated_r_c <= r_slices[max_index[0] + n]) & (event.rotated_r_c >= r_slices[max_index[0] - n])]
  cut_event = event[(event.t_c <= np.clip(t_slices[max_index[1]] + t_window_width + timestep * m, 0, 10)) & (event.t_c >= np.clip(t_slices[max_index[1]] - timestep * m, 0, 10)) &
                    (event.rotated_r_c <= r_slices[max_index[0] + n]) & (event.rotated_r_c >= r_slices[max_index[0] - n])]
  return np.mean(cut_event.rotated_r_c)

In [28]:
def local_weighed_sum_2d(r_slices, t_slices, square_counts, max_index, n, m, method='N/r'):
  arr = np.mean(square_counts[max_index[0] - n:max_index[0] + n + 1, np.clip(max_index[1] - m, 0, 50):np.clip(max_index[1] + m + 1, 0, 50)], axis=1)
  if method == 'N/r':
    sum_arr = r_slices[max_index[0] - n:max_index[0] + n + 1] ** 2 * arr
    den_arr = r_slices[max_index[0] - n:max_index[0] + n + 1] * arr
  else:
    sum_arr = r_slices[max_index[0] - n:max_index[0] + n + 1] * arr
    den_arr = arr

  weighted_sum = np.sum(sum_arr)
  weighted_den = np.sum(den_arr)

  return weighted_sum / weighted_den

In [29]:
def local_weighed_sum(r_slices, counts, max_index, n, method='N/r'):
  if method == 'N/r':
    sum_arr = r_slices[max_index - n:max_index + n + 1] ** 2 * counts[max_index - n:max_index + n + 1]
    den_arr = r_slices[max_index - n:max_index + n + 1] * counts[max_index - n:max_index + n + 1]
  else:
    sum_arr = r_slices[max_index - n:max_index + n + 1] * counts[max_index - n:max_index + n + 1]
    den_arr = counts[max_index - n:max_index + n + 1]

  weighted_sum = np.sum(sum_arr)
  weighted_den = np.sum(den_arr)

  return weighted_sum / weighted_den

In [30]:
# for unweighed with t sigms
cal_arr = [[[ 9.67217302e-01,  2.09919513e+01,  2.67663027e+02],
        [ 9.63308753e-01,  9.28031497e+00,  3.11704621e+02],
        [ 9.62251886e-01,  7.53836278e+00,  3.05891956e+02],
        [ 9.58121999e-01, -2.58516699e+00,  3.33736988e+02],
        [ 9.58954093e-01, -1.04163651e+00,  3.23052716e+02],
        [ 9.52278216e-01, -1.47044399e+01,  3.56504808e+02],
        [ 9.54017604e-01, -1.21125683e+01,  3.46156270e+02],
        [ 9.50380251e-01, -1.83659195e+01,  3.54173368e+02],
        [ 9.54750950e-01, -6.94199414e+00,  3.09830431e+02],
        [ 9.54404052e-01, -6.29685848e+00,  2.99013016e+02]],

       [[ 9.67232869e-01,  2.10733318e+01,  2.67664993e+02],
        [ 9.63676285e-01,  1.04065598e+01,  3.06991663e+02],
        [ 9.62198143e-01,  7.36510150e+00,  3.07196337e+02],
        [ 9.59032260e-01, -4.94297471e-01,  3.27323135e+02],
        [ 9.59644186e-01,  6.64838843e-01,  3.17387097e+02],
        [ 9.53551693e-01, -1.16744906e+01,  3.47314177e+02],
        [ 9.54052701e-01, -1.20174560e+01,  3.46211312e+02],
        [ 9.50963409e-01, -1.70591122e+01,  3.50758741e+02],
        [ 9.55067684e-01, -6.31241141e+00,  3.08343530e+02],
        [ 9.54576657e-01, -5.97157441e+00,  2.98271817e+02]],

        [[ 9.67286549e-01,  2.12872046e+01,  2.66312851e+02],
        [ 9.63680207e-01,  1.04154429e+01,  3.06957562e+02],
        [ 9.62193754e-01,  7.34831699e+00,  3.07241817e+02],
        [ 9.59070143e-01, -3.80647018e-01,  3.26838619e+02],
        [ 9.59705455e-01,  8.11853641e-01,  3.16895628e+02],
        [ 9.53517313e-01, -1.17447615e+01,  3.47502116e+02],
        [ 9.54369357e-01, -1.13378424e+01,  3.44369555e+02],
        [ 9.51317881e-01, -1.63012731e+01,  3.48654656e+02],
        [ 9.55101151e-01, -6.22860310e+00,  3.08048969e+02],
        [ 9.54602516e-01, -5.91148966e+00,  2.98057158e+02]],

       [[ 9.67428846e-01,  2.17458415e+01,  2.64202459e+02],
        [ 9.63645215e-01,  1.03212472e+01,  3.07492834e+02],
        [ 9.62616823e-01,  8.61684113e+00,  3.02186055e+02],
        [ 9.59508289e-01,  6.66027725e-01,  3.23443037e+02],
        [ 9.59553137e-01,  2.71897075e-01,  3.19371628e+02],
        [ 9.53610322e-01, -1.15451342e+01,  3.47105881e+02],
        [ 9.54410706e-01, -1.12152614e+01,  3.44082727e+02],
        [ 9.50908322e-01, -1.72827756e+01,  3.51726891e+02],
        [ 9.54714914e-01, -7.02420675e+00,  3.10522667e+02],
        [ 9.54838829e-01, -5.47322202e+00,  2.96813889e+02]]]

In [31]:
# # for t sigms
# cal_arr = [[[  0.96348891,   9.47067408, 312.33234386],
#         [  0.96347449,   8.35497804, 311.38699743],
#         [  0.96022277,   0.64583334, 329.22824922],
#         [  0.94999758, -22.71606321, 391.16523552],
#         [  0.95625113,  -9.36401448, 349.40428664],
#         [  0.94257101, -37.97604321, 417.58906762],
#         [  0.9497489 , -22.84759782, 374.13975064],
#         [  0.95383741, -11.93800769, 330.76314339],
#         [  0.95371146,  -9.87478762, 311.60505387],
#         [  0.95022252, -15.28875862, 316.65353399]],
#
#        [[  0.96348869,   9.47801582, 312.24120779],
#         [  0.96345217,   8.30636783, 311.5619668 ],
#         [  0.96022344,   0.64484706, 329.21489154],
#         [  0.95010622, -22.43305616, 390.28342888],
#         [  0.95625466,  -9.34908595, 349.33432219],
#         [  0.94251642, -38.11068835, 417.94983549],
#         [  0.94975533, -22.83262017, 374.10111702],
#         [  0.95382051, -11.97762055, 330.89163801],
#         [  0.95370019,  -9.89445279, 311.64975442],
#         [  0.95020812, -15.3205052 , 316.74814168]],
#
#         [[  0.96347485,   9.43939839, 312.39556339],
#         [  0.96345644,   8.31040864, 311.56343202],
#         [  0.96023336,   0.67249674, 329.10560665],
#         [  0.95015702, -22.30681397, 389.91246634],
#         [  0.95627437,  -9.31280473, 349.25933064],
#         [  0.94261153, -37.88854044, 417.38411568],
#         [  0.95015564, -21.96280019, 371.83747033],
#         [  0.95413409, -11.29697304, 328.87396083],
#         [  0.9537581 ,  -9.77268937, 311.28244172],
#         [  0.95026042, -15.21548413, 316.43465242]],
#
#        [[  0.96366797,   9.9276347 , 310.79705191],
#         [  0.96349078,   8.43960589, 311.06738539],
#         [  0.96104111,   2.85058786, 321.28032786],
#         [  0.95106956, -20.14692625, 383.91002168],
#         [  0.95610093,  -9.9550001 , 352.05129   ],
#         [  0.9421615 , -38.94929249, 420.33244976],
#         [  0.95106024, -19.84169775, 365.72286181],
#         [  0.95396477, -11.76170309, 330.5511635 ],
#         [  0.95373554,  -9.86091787, 311.68900672],
#         [  0.95065609, -14.52018021, 314.63980829]]]

In [32]:
# for R sigms
# cal_arr = [[[ 9.61663804e-01,  4.51654847e+00,  3.35099975e+02],
#         [ 9.59125687e-01, -2.06329300e+00,  3.49722028e+02],
#         [ 9.57936042e-01, -4.25686977e+00,  3.46380859e+02],
#         [ 9.48728410e-01, -2.64885477e+01,  4.06502821e+02],
#         [ 9.40144120e-01, -4.65708970e+01,  4.52032887e+02],
#         [ 9.22028246e-01, -8.77789300e+01,  5.35260817e+02],
#         [ 7.43713563e-01, -5.32545455e+02,  1.09630968e+03],
#         [ 8.46514571e-01, -2.51247655e+02,  7.67673373e+02],
#         [ 9.37974939e-01, -4.24201841e+01,  4.02663910e+02],
#         [ 9.47261851e-01, -2.05714504e+01,  3.34227253e+02]],
#
#        [[ 9.61347834e-01,  3.98397062e+00,  3.37139353e+02],
#         [ 9.59263504e-01, -1.35920441e+00,  3.47255358e+02],
#         [ 9.58868958e-01, -1.38033689e+00,  3.35896872e+02],
#         [ 9.51559550e-01, -1.89686781e+01,  3.83999986e+02],
#         [ 9.44778245e-01, -3.46941082e+01,  4.20565420e+02],
#         [ 9.29807803e-01, -6.84999868e+01,  4.93328721e+02],
#         [ 8.56513287e-01, -2.35749556e+02,  7.61034612e+02],
#         [ 8.99275176e-01, -1.28514352e+02,  5.84164266e+02],
#         [ 9.43640651e-01, -2.99069921e+01,  3.70261857e+02],
#         [ 9.48317539e-01, -1.79808055e+01,  3.26054458e+02]],
#
#        [[ 9.60940227e-01,  3.54186693e+00,  3.38467974e+02],
#         [ 9.60116759e-01,  1.53717740e+00,  3.36683346e+02],
#         [ 9.59541043e-01,  8.83800395e-01,  3.28020773e+02],
#         [ 9.54548349e-01, -1.09694205e+01,  3.59173051e+02],
#         [ 9.50110825e-01, -2.13031697e+01,  3.83288194e+02],
#         [ 9.41089670e-01, -4.12804910e+01,  4.28795205e+02],
#         [ 9.17030639e-01, -9.26876304e+01,  5.30338207e+02],
#         [ 9.33527159e-01, -5.28113100e+01,  4.35497726e+02],
#         [ 9.48632748e-01, -1.89101523e+01,  3.39582895e+02],
#         [ 9.48980996e-01, -1.63407625e+01,  3.20806451e+02]],
#
#        [[ 9.61571328e-01,  5.73918236e+00,  3.31189141e+02],
#         [ 9.60299477e-01,  2.57060566e+00,  3.33725638e+02],
#         [ 9.60190718e-01,  3.19760353e+00,  3.20195506e+02],
#         [ 9.57078419e-01, -4.21032356e+00,  3.37898366e+02],
#         [ 9.53079362e-01, -1.34991832e+01,  3.60295567e+02],
#         [ 9.46899320e-01, -2.72331215e+01,  3.92509043e+02],
#         [ 9.39294061e-01, -4.21593004e+01,  4.20802723e+02],
#         [ 9.45649894e-01, -2.62275862e+01,  3.69624662e+02],
#         [ 9.50241995e-01, -1.50173044e+01,  3.27591656e+02],
#         [ 9.49667036e-01, -1.49467584e+01,  3.16823657e+02]]]

In [33]:
def pol(x, a, b, c):
  return a * np.exp((x - b) ** 2 / c ** 2)

In [34]:
def rSlidingWindowIntro(edf: pd.DataFrame, idf: pd.DataFrame, bdf: pd.DataFrame, avg_sigmas: tuple, avg_t_sigmas: tuple, step=float(idf.pixel_size), method='N/r', cal_arr=cal_arr, t_window_width=2,
                        r_width_factor=2, t_width_factor=8, full_width_t_hist = False, num_of_groups=5):
  r_r_c = edf['rotated_r_c']
  time_step = float(t_window_width) / t_width_factor
  all_avgs = np.array(r_r_c.groupby(level=0).transform('mean').to_list()).ravel()
  all_dists = np.abs(r_r_c - all_avgs)
  all_sigms = np.array(r_r_c.groupby(level=0).transform('std').to_list()).ravel()

  edf['mean_rotated_r_c'] = all_avgs
  edf['dist_from_mean_rotated_r_c'] = all_dists
  edf['rotated_r_c_sigm'] = all_sigms

  # Compute beta_step and r_step using NumPy functions
  beta_step = np.ptp(edf['beta'].values) # не факт что нужно values
  r_step = np.ptp(edf['true_r_from_beta'].values)

  # Compute beta_intervals using NumPy linspace function
  num_of_groups = num_of_groups
  beta_intervals = np.linspace(edf['beta'].min(), edf['beta'].max(), num=num_of_groups)

  # Compute beta_group_to_bdf and true_r_group using NumPy operations
  beta_group = np.floor((num_of_groups * edf['beta'] + max(edf['beta']) - (num_of_groups + 1) * min(edf['beta'])) / beta_step).values
  true_r_group = np.floor((num_of_groups * edf['true_r_from_beta'] + max(edf['true_r_from_beta']) - (num_of_groups + 1) * min(edf['true_r_from_beta'])) / r_step).values

  edf['beta_group'] = beta_group
  edf['true_r_group'] = true_r_group

  edf_to_bdf(edf.beta_group, bdf)
  edf_to_bdf(edf.true_r_group, bdf)

  edf_to_bdf(edf.theta_p, bdf)
  bdf['cos_theta_p'] = np.cos(bdf['theta_p'])
  # edf_to_bdf(edf.signal_counts, bdf)
  edf_to_bdf(edf.beta, bdf)
  edf_to_bdf(edf.true_r_from_beta, bdf)

  # edf['slice_group'] = np.floor(r_r_c / step) * step
  edf['slice_group'] = r_r_c # test
  # edf['time_slice_group'] = np.floor(edf.t_c / time_step) * time_step
  edf['time_slice_group'] = edf.t_c # test2


In [35]:
def calculateSignalCounts(edf: pd.DataFrame, bdf: pd.DataFrame):
  signal_counts = edf['signal'].groupby(level=0).sum()
  bdf['signal_counts'] = signal_counts.values
  edf['signal_counts'] = edf.signal.groupby(level=0).transform('sum').values

In [36]:
def rSlidingWindowLoop1Unfixed(edf: pd.DataFrame, idf: pd.DataFrame, bdf: pd.DataFrame, avg_sigmas: tuple, avg_t_sigmas: tuple, step=float(idf.pixel_size), method='N/r', cal_arr=cal_arr, timestep=0.25):
  step = step / 2
  time_step = timestep

  r_slices = np.arange(0, 800, step=step)
  t_slices = np.arange(0, 10, step=time_step)

  all_counts_to_edf = []
  all_calculated_r = []

  for _ in range(np.ptp(avg_sigmas)):
    all_counts_to_edf.append([])
    all_calculated_r.append([])

  for entry, subentry in edf[['slice_group', 'time_slice_group']].groupby(level=0):
    counts = np.zeros(r_slices.shape)
    square_counts = np.zeros(shape=(r_slices.shape[0], t_slices.shape[0]))
    for _, subsubenrty in subentry.iterrows():
      if 80 >= subsubenrty.slice_group >= 16:  # Тут было ограничение сверху 70, снизу 10
        counts[int(np.round(float(subsubenrty.slice_group) / step, 0))] += 1 # а тут floor а не round
        square_counts[int(np.round(float(subsubenrty.slice_group) / step, 0)), int(np.round(float(subsubenrty.time_slice_group) / time_step, 0))] += 1

    if method == 'N/r':
      counts = np.array([(counts[i - 1] + counts[i]) / r_slices[i] for i in range(1, counts.shape[0])])
      for i in range(square_counts.shape[0] - 1):
        square_counts[i, :] = np.array((square_counts[i, :] + square_counts[i + 1, :]) / r_slices[i + 1])

    max_index = np.where(counts == max(counts))[0][0]

    for n_sigms in range(*avg_sigmas):
      avg_r_from_slices = local_weighed_sum(r_slices, counts, max_index, n_sigms, method)
      # counts_to_edf = []
      for _, subsubenrty in subentry.iterrows():
        all_counts_to_edf[n_sigms - avg_sigmas[0]].append(counts[int(np.floor(float(subsubenrty.slice_group) / step))])
        all_calculated_r[n_sigms - avg_sigmas[0]].append(avg_r_from_slices)
  for n_sigms in range(*avg_sigmas):
    edf[f'slice_counts_{n_sigms}_sigms'] = all_counts_to_edf[n_sigms - avg_sigmas[0]]
    edf[f'unfixed_calculated_r_{n_sigms}_sigms'] = all_calculated_r[n_sigms - avg_sigmas[0]]

In [37]:
def rSlidingWindowLoop1(edf: pd.DataFrame, idf: pd.DataFrame, bdf: pd.DataFrame, avg_sigmas: tuple, avg_t_sigmas: tuple, step=float(idf.pixel_size), method='N/r', cal_arr=cal_arr, t_window_width=2,
                        r_width_factor=2, t_width_factor=8, full_width_t_hist = True, weighed = True):
  step = step / r_width_factor
  time_step = float(t_window_width) / t_width_factor

  r_slices = np.arange(0, 800, step=step)
  t_slices = np.arange(0, 15, step=time_step)

  n_sigmas = np.ptp(avg_sigmas)
  t_sigmas = np.ptp(avg_t_sigmas)
  # all_counts_to_edf = np.zeros((n_sigmas, len(edf)))
  all_counts_to_edf = np.zeros((n_sigmas, len(edf)))
  all_calculated_r = np.zeros((n_sigmas, len(edf)))
  all_calculated_r_from_2d = np.zeros((t_sigmas, n_sigmas, len(edf)))
  cur_ind = 0
  for i, (entry, subentry) in enumerate(edf[['slice_group', 'time_slice_group', 'rotated_r_c', 't_c']].groupby(level=0)):
    counts = np.zeros(r_slices.shape)
    square_counts = np.zeros(shape=(r_slices.shape[0], t_slices.shape[0]))

    mask = np.logical_and(subentry.slice_group >= 16, subentry.slice_group <= 80)
    slice_groups = subentry.slice_group[mask]
    time_slice_groups = subentry.time_slice_group[mask]

    counts, _ = np.histogram(slice_groups, bins=r_slices)
    square_counts, _, __ = np.histogram2d(slice_groups, time_slice_groups, bins=(r_slices, t_slices))

    if method == 'N/r':
      counts = np.divide(np.add(counts[:-1], counts[1:]), r_slices[1:-1])
      square_counts[:-1, :] = np.divide(np.add(square_counts[:-1, :], square_counts[1:, :]), r_slices[1:-1, np.newaxis])
    if full_width_t_hist:
      square_counts_but_last = sum([square_counts[:, it : -t_width_factor + 1 + it] for it in range(t_width_factor - 1)])
      square_counts = np.add(square_counts_but_last, square_counts[:, t_width_factor - 1:])

    max_index = np.argmax(counts)

    max_index_2d = np.unravel_index(np.argmax(square_counts), square_counts.shape)

    for j in range(n_sigmas):
      all_counts_to_edf[j][cur_ind:subentry.shape[0] + cur_ind] = counts[np.floor_divide(subentry.slice_group, step).astype(int)] # fixed
      # avg_r_from_slices = local_weighed_sum(r_slices, counts, max_index, j + avg_sigmas[0], method)
      # all_calculated_r[j, cur_ind:subentry.shape[0] + cur_ind] = np.repeat(avg_r_from_slices, subentry.shape[0])
      for t in range(t_sigmas):
        if weighed:
          avg_r_from_2d_slices = local_weighed_sum_2d(r_slices, t_slices, square_counts, max_index_2d, j + avg_sigmas[0], t + avg_t_sigmas[0])
        else:
          avg_r_from_2d_slices = local_sum_2d(subentry, r_slices, t_slices, square_counts, max_index_2d, j + avg_sigmas[0], t + avg_t_sigmas[0], t_window_width=t_window_width, timestep=time_step)

        # if np.isnan(avg_r_from_2d_slices):
        #   print(max_index_2d)
        #   print(square_counts[max_index_2d[0] - 1:max_index_2d[0] + 2, max_index_2d[1] - 1: max_index_2d[1] + 2])
        #   raise ValueError
        all_calculated_r_from_2d[t, j, cur_ind:subentry.shape[0] + cur_ind] = np.repeat(avg_r_from_2d_slices, subentry.shape[0])

    cur_ind += subentry.shape[0]
  for j in range(n_sigmas):
    edf[f'slice_counts_{j + avg_sigmas[0]}_sigms'] = all_counts_to_edf[j]
    # edf[f'unfixed_calculated_r_{j + avg_sigmas[0]}_sigms'] = all_calculated_r[j, :]
    for t in range(t_sigmas):
      edf[f'unfixed_calculated_r_2d_{j + avg_sigmas[0]}_rsigms_{t + avg_t_sigmas[0]}_tsigms'] = all_calculated_r_from_2d[t, j, :]
      edf_to_bdf(edf[f'unfixed_calculated_r_2d_{j + avg_sigmas[0]}_rsigms_{t + avg_t_sigmas[0]}_tsigms'], bdf)

In [38]:
def rSlidingWindowLoop2(edf: pd.DataFrame, idf: pd.DataFrame, bdf: pd.DataFrame, avg_sigmas: tuple, avg_t_sigmas: tuple, step=float(idf.pixel_size), method='N/r', cal_arr=cal_arr, t_window_width=2,
                        r_width_factor=2, t_width_factor=8, full_width_t_hist = False, param_fit=False):

  # cal_arr = np.array([np.array([np.array(y) for y in x]) for x in cal_arr])

  edf_to_bdf(edf.theta_p, bdf)
  bdf['cos_theta_p'] = np.cos(bdf['theta_p'])
  edf['cos_theta_p'] = np.cos(edf['theta_p'])
  theta_interval = np.ptp(bdf.cos_theta_p) / 10
  theta_min = min(bdf.cos_theta_p)
  theta_max = max(bdf.cos_theta_p)
  edf_to_bdf(edf.beta, bdf)
  edf_to_bdf(edf.true_r_from_beta, bdf)

  # for n_sigms in range(*avg_sigmas):
  #   meas_betas = []
  #   for entry, subentry in edf[f'unfixed_calculated_r_{n_sigms}_sigms'].groupby(level=0):
  #     try:
  #       meas_beta = pol(subentry.iloc[0], *(cal_arr[n_sigms - 1][(np.floor((np.cos(edf.theta_p[entry].iloc[0]) - theta_min) / (theta_interval))).astype(int)]))
  #     except IndexError:
  #       meas_beta = pol(subentry.iloc[0], *(cal_arr[n_sigms - 1][9]))
  #       print((np.cos(edf.theta_p[entry].iloc[0]) - theta_min) / (theta_interval))
  #     for subsub in subentry:
  #       meas_betas.append(meas_beta)
  #   edf[f'beta_from_calc_r_{n_sigms}_sigms'] = meas_betas
  #
  #   edf[f'delta_beta_{n_sigms}_sigms'] = edf[f'beta_from_calc_r_{n_sigms}_sigms'] - edf['beta']
  #   edf[f'eps_beta_{n_sigms}_sigms'] = edf[f'delta_beta_{n_sigms}_sigms'] / edf['beta'] * 100
  #
  #   edf_to_bdf(edf[f'unfixed_calculated_r_{n_sigms}_sigms'], bdf)
  #   edf_to_bdf(edf[f'beta_from_calc_r_{n_sigms}_sigms'], bdf)
  #   edf_to_bdf(edf[f'delta_beta_{n_sigms}_sigms'], bdf)
  #   edf_to_bdf(edf[f'eps_beta_{n_sigms}_sigms'], bdf)


  for n_sigms in range(*avg_sigmas):
    for t_sigms in range(*avg_t_sigmas):
      meas_betas = np.zeros(edf.shape[0])
      cur_ind = 0
      # meas_betas = []
      for entry, subentry in edf[f'unfixed_calculated_r_2d_{n_sigms}_rsigms_{t_sigms}_tsigms'].groupby(level=0):
        if param_fit:
          cos_theta_p = edf.cos_theta_p[entry].iloc[0]
          meas_beta = pol(subentry.iloc[0], pol2(cos_theta_p, *cal_arr[n_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][0]),
                          pol2(cos_theta_p, *cal_arr[n_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][1]),
                          pol2(cos_theta_p, *cal_arr[n_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][2]))
        else:
          if edf.cos_theta_p[entry].iloc[0] != theta_max:
            meas_beta = pol(subentry.iloc[0], *(cal_arr[n_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][(np.floor(((edf.cos_theta_p[entry].iloc[0]) - theta_min) / theta_interval)).astype(int)]))
          else:
            meas_beta = pol(subentry.iloc[0], *(cal_arr[n_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][9]))
        meas_betas[cur_ind: subentry.shape[0] + cur_ind] = np.repeat(meas_beta, subentry.shape[0])
        cur_ind += subentry.shape[0]
        # for subsub in subentry:
        #   meas_betas.append(meas_beta)  # FIXME
      edf[f'beta_from_calc_r_{n_sigms}_rsigms_{t_sigms}_tsigms'] = meas_betas
      edf[f'delta_beta_{n_sigms}_rsigms_{t_sigms}_tsigms'] = edf[f'beta_from_calc_r_{n_sigms}_rsigms_{t_sigms}_tsigms'] - edf['beta']
      edf[f'eps_beta_{n_sigms}_rsigms_{t_sigms}_tsigms'] = edf[f'delta_beta_{n_sigms}_rsigms_{t_sigms}_tsigms'] / edf['beta'] * 100


      edf_to_bdf(edf[f'beta_from_calc_r_{n_sigms}_rsigms_{t_sigms}_tsigms'], bdf)
      edf_to_bdf(edf[f'delta_beta_{n_sigms}_rsigms_{t_sigms}_tsigms'], bdf)
      edf_to_bdf(edf[f'eps_beta_{n_sigms}_rsigms_{t_sigms}_tsigms'], bdf)

In [39]:
def rSlidingWindow(edf: pd.DataFrame, idf: pd.DataFrame, bdf: pd.DataFrame, avg_sigmas: tuple, avg_t_sigmas: tuple, step=float(idf.pixel_size), method='N/r', cal_arr=False, t_window_width=2,
                        r_width_factor=2, t_width_factor=8, full_width_t_hist = True, num_of_groups=5, weighed=True, deg_lim=False, param_fit=False):
  rSlidingWindowIntro(edf, idf, bdf, avg_sigmas, avg_t_sigmas, step=step, method=method, cal_arr=cal_arr, t_window_width=t_window_width,
                      r_width_factor=r_width_factor, t_width_factor=t_width_factor, num_of_groups=num_of_groups)
  calculateSignalCounts(edf, bdf)
  rSlidingWindowLoop1(edf, idf, bdf, avg_sigmas, avg_t_sigmas, step=step, method=method, cal_arr=cal_arr, t_window_width=t_window_width,
                      r_width_factor=r_width_factor, t_width_factor=t_width_factor, full_width_t_hist=full_width_t_hist, weighed=weighed)
  if cal_arr is False:
    cal_arr = np.array(calibration(edf, bdf, avg_sigmas=avg_sigmas, avg_t_sigmas=avg_t_sigmas, step=step, t_window_width=t_window_width,
                                   r_width_factor=r_width_factor, t_width_factor=t_width_factor, weighed=weighed, deg_lim=deg_lim, param_fit=param_fit)) # add r and t calibr - done
  rSlidingWindowLoop2(edf, idf, bdf, avg_sigmas, avg_t_sigmas, step=step, method=method, cal_arr=cal_arr, t_window_width=t_window_width,
                      r_width_factor=r_width_factor, t_width_factor=t_width_factor, param_fit=param_fit)
  return cal_arr

In [40]:
def rSlidingWindowOld(edf: pd.DataFrame, idf:pd.DataFrame, bdf:pd.DataFrame, step=float(idf['pixel_size']), method='N/r', cal_arr=cal_arr):
  r_r_c = edf['rotated_r_c'].to_numpy()
  all_counts = []
  all_avgs = []
  all_dists = []
  all_sigms = []
  theta_interval = np.ptp(np.cos(edf.theta_p)) / 10
  theta_min = min(np.cos(edf.theta_p))
  cal_arr = np.array([np.array([np.array(y) for y in x]) for x in cal_arr])
  for entry, subentry in edf['rotated_r_c'].groupby(level=0):
    avg = np.average(subentry)
    disp = np.std(subentry)
    for subsubentry in subentry:
      all_avgs.append(avg)
      all_dists.append(np.abs(avg - subsubentry))
      all_sigms.append(disp)

  all_avgs = np.array(all_avgs)
  all_dists = np.array(all_dists)
  all_sigms = np.array(all_sigms)

  edf['mean_rotated_r_c'] = all_avgs
  edf['dist_from_mean_rotated_r_c'] = all_dists
  edf['rotated_r_c_sigm'] = all_sigms

  # print(np.average(all_sigms))

  beta_step = float((max(edf['beta']) - min(edf['beta'])))
  r_step = float((max(edf['true_r_from_beta']) - min(edf['true_r_from_beta'])))
  beta_intervals = np.linspace(min(edf['beta']), max(edf['beta']), num=5)
  beta_group_to_bdf = []
  true_r_group = []
  edf['true_r_group'] = np.floor((5 * edf['true_r_from_beta'] + max(edf['true_r_from_beta']) - 6 * min(edf['true_r_from_beta'])) / r_step)
  edf['beta_group'] = np.floor((5 * edf['beta'] + max(edf['beta']) - 6 * min(edf['beta'])) / beta_step)
  for entry, subentry in edf['beta_group'].groupby(level=0):
    for subsubentry in subentry:
      beta_group_to_bdf.append(subsubentry)
      break
  for entry, subentry in edf['true_r_group'].groupby(level=0):
    for subsubentry in subentry:
      true_r_group.append(subsubentry)
      break

  bdf['beta_group'] = pd.Series(beta_group_to_bdf)
  bdf['true_r_group'] = pd.Series(true_r_group)
  step = step / 2
  time_step = 0.25
  # step = float(idf['pixel_size'])
  # step = np.average(all_sigms)
  r_slices = np.arange(0, 800, step=step)
  t_slices = np.arange(0, 10, step=time_step)

  def local_weighed_sum(r_slices, counts, max_index, n, method='N/r'): # n = 1 -> +-0.5 sigm, n = 2 -> +-1 sigm...
    if method == 'N/r':
      sum =  r_slices[max_index] * r_slices[max_index] * counts[max_index]
      den = r_slices[max_index] * counts[max_index]
      for i in range(1, n + 1):
        sum +=  r_slices[max_index - i] * r_slices[max_index - i] * counts[max_index - i] +  r_slices[max_index + i] * r_slices[max_index + i] * counts[max_index + i]
        den +=  r_slices[max_index - i] * counts[max_index - i] + r_slices[max_index + i] * counts[max_index + i]
    else:
      sum = r_slices[max_index] * counts[max_index]
      den = counts[max_index]
      for i in range(1, n + 1):
        sum +=  r_slices[max_index - i] * counts[max_index - i] +  r_slices[max_index + i] * counts[max_index + i]
        den +=  counts[max_index - i] + counts[max_index + i]
    return sum / den

  def r_from_correction(x, n=1.0454, l=200, dl=17.5):
    return x * (1 + n * l / np.sqrt(dl ** 2 - x ** 2 * (1 - n ** 2)))

  def r_from_beta_and_correction(beta, n=1.0454, l=200, dl=17.5, n_max=1.05):
    return dl * np.tan(np.arccos(1 / beta / n)) + l * np.tan(np.arcsin(n_max * np.sqrt(1 - 1 / (beta * n ) ** 2)))

  r_correction_from_r = inversefunc(r_from_correction)
  r_interval = np.linspace(1, 100, num=200)
  r_correction_on_interval = r_correction_from_r(r_interval)
  # beta_from_r = inversefunc(r_from_beta_and_correction, domain=[0.95, 1])
  # beta_on_interval = beta_from_r(r_interval)

  true_r_from_beta = np.array(edf['true_r_from_beta'])
  # edf['corrected_true_r_from_beta'] = edf['true_r_from_beta'] + r_correction_on_interval[(true_r_from_beta / (r_interval[1] - r_interval[0])).astype(int)]
  edf['corrected_true_r_from_beta'] = r_from_beta_and_correction(edf['beta'])

  edf['slice_group'] = np.floor(r_r_c / step) * step
  edf['time_slice_group'] = np.floor(edf.t_c / time_step) * time_step
  for n_sigms in range(4, 5):
    all_counts_to_edf = []
    all_calculated_r = []
    all_calculated_r_fixed = []
    for entry, subentry in edf[['slice_group', 'time_slice_group']].groupby(level=0):
      counts = np.zeros(r_slices.shape)
      square_counts = np.zeros(shape=(r_slices.shape[0], t_slices.shape[0]))
      for _, subsubenrty in subentry.iterrows():
        if 80 >= subsubenrty.slice_group >= 16:  # Тут было ограничение сверху 70, снизу 10
          counts[int(np.round(float(subsubenrty.slice_group) / step, 0))] += 1 # а тут floor а не round
          square_counts[int(np.round(float(subsubenrty.slice_group) / step, 0)), int(np.round(float(subsubenrty.time_slice_group) / time_step, 0))] += 1


      if method == 'N/r':
        counts = np.array([(counts[i - 1] + counts[i]) / r_slices[i] for i in range(1, counts.shape[0])])
        for i in range(0, square_counts.shape[0] - 1):
          square_counts[i, :] = np.array((square_counts[i, :] + square_counts[i + 1, :]) / r_slices[i + 1])
        # square_counts[,:] = np.array([(square_counts[i - 1, :] + counts[i, :]) / r_slices[i] for i in range(1, square_counts.shape[0][0])])
        # counts = np.array([(counts[i]) / r_slices[i] for i in range(1, counts.shape[0])])

      # max_index = counts.index(max(counts))
      max_index = np.where(counts == max(counts))[0][0]


      avg_r_from_slices = local_weighed_sum(r_slices, counts, max_index, n_sigms, method)
      # counts_to_edf = []
      for _, subsubenrty in subentry.iterrows():
        all_counts_to_edf.append(counts[int(np.floor(float(subsubenrty.slice_group) / step))])
        all_calculated_r.append(avg_r_from_slices)
        if np.isnan(avg_r_from_slices):
          all_calculated_r_fixed.append(avg_r_from_slices  * (1 - 0.0775))
        else:
          all_calculated_r_fixed.append(avg_r_from_slices - r_correction_on_interval[int(avg_r_from_slices / (r_interval[1] - r_interval[0]))])

    signal_counts_to_bdf= []
    signal_counts_to_edf = []
    for _, subentry in edf['signal'].groupby(level=0):
      try:
        signal_counts = subentry.value_counts()[True]
      except KeyError:
        signal_counts = 0
        print(subentry.value_counts())
      signal_counts_to_bdf.append(signal_counts)
      for subsubenrty in subentry:
        signal_counts_to_edf.append(signal_counts)

    bdf['signal_counts'] = pd.Series(signal_counts_to_bdf)
    edf['signal_counts'] = signal_counts_to_edf

    edf[f'slice_counts_{n_sigms}_sigms'] = all_counts_to_edf

    edf[f'unfixed_calculated_r_{n_sigms}_sigms'] = all_calculated_r
    edf[f'calculated_r_{n_sigms}_sigms'] = all_calculated_r_fixed
    edf[f'delta_calc_true_r_{n_sigms}_sigms'] = edf[f'calculated_r_{n_sigms}_sigms'] - edf['true_r_from_beta']
    edf[f'eps_calc_true_r_{n_sigms}_sigms'] = edf[f'delta_calc_true_r_{n_sigms}_sigms'] / edf['true_r_from_beta'] * 100

    # edf[f'beta_from_calc_r_{n_sigms}_sigms'] = np.sqrt((edf[f'calculated_r_{n_sigms}_sigms']) ** 2 + ((float(idf['distance']) + float(idf['W']) / 2) ** 2)) / (float(idf['n_mean']) * (float(idf['distance']))+float(idf['W'])/2)
    # edf[f'beta_from_calc_r_{n_sigms}_sigms'] = beta_on_interval[int(all_calculated_r / (r_interval[1] - r_interval[0]))]
    meas_betas = []
    for entry, subentry in edf[f'unfixed_calculated_r_{n_sigms}_sigms'].groupby(level=0):
      try:
        meas_beta = pol(subentry.iloc[0], *(cal_arr[n_sigms - 1][(np.floor((np.cos(edf.theta_p[entry].iloc[0]) - theta_min) / (theta_interval))).astype(int)]))
      except IndexError:
        meas_beta = pol(subentry.iloc[0], *(cal_arr[n_sigms - 1][9]))
        print((np.cos(edf.theta_p[entry].iloc[0]) - theta_min) / (theta_interval))
      for subsub in subentry:
        meas_betas.append(meas_beta)
    edf[f'beta_from_calc_r_{n_sigms}_sigms'] = meas_betas
    # edf[f'beta_from_calc_r_{n_sigms}_sigms'] = pol(edf[f'unfixed_calculated_r_{n_sigms}_sigms'], *(cal_arr[(np.floor((np.cos(edf.theta_p) - theta_min) / theta_interval)).astype(int) - 1]))

    edf[f'delta_beta_{n_sigms}_sigms'] = edf[f'beta_from_calc_r_{n_sigms}_sigms'] - edf['beta']
    edf[f'eps_beta_{n_sigms}_sigms'] = edf[f'delta_beta_{n_sigms}_sigms'] / edf['beta'] * 100


    edf_to_bdf(edf.theta_p, bdf)
    bdf['cos_theta_p'] = np.cos(bdf['theta_p'])
    # edf_to_bdf(edf.signal_counts, bdf)
    edf_to_bdf(edf.beta, bdf)
    edf_to_bdf(edf.true_r_from_beta, bdf)
    edf_to_bdf(edf.corrected_true_r_from_beta, bdf)

    edf_to_bdf(edf[f'beta_from_calc_r_{n_sigms}_sigms'], bdf)
    edf_to_bdf(edf[f'delta_beta_{n_sigms}_sigms'], bdf)
    edf_to_bdf(edf[f'eps_beta_{n_sigms}_sigms'], bdf)
    edf_to_bdf(edf[f'unfixed_calculated_r_{n_sigms}_sigms'], bdf)
    edf_to_bdf(edf[f'calculated_r_{n_sigms}_sigms'], bdf)
    edf_to_bdf(edf[f'delta_calc_true_r_{n_sigms}_sigms'], bdf)
    edf_to_bdf(edf[f'eps_calc_true_r_{n_sigms}_sigms'], bdf)

  # all_counts_to_edf = np.array(all_counts_to_edf)
  edf['slice_counts'] = all_counts_to_edf
  # edf['calculated_r'] = all_calculated_r
  # edf['delta_calc_true_r'] = np.abs(edf['calculated_r'] - edf['true_r_from_beta'])
  # edf['eps_calc_true_r'] = edf['delta_calc_true_r'] / edf['calculated_r'] *

  # edf['beta_from_calc_r'] = np.sqrt((edf['calculated_r']) ** 2 + (edf['r_p_c']) ** 2) / (float(idf['n_mean']) * edf['r_p_c'])
  # edf['delta_beta'] = np.abs(edf['beta_from_calc_r'] - edf['beta'])
  # edf['eps_beta'] = edf['delta_beta'] / edf['beta'] * 100

In [41]:
def rSlidingWindowTest(edf: pd.DataFrame, idf:pd.DataFrame, bdf:pd.DataFrame, step=float(idf['pixel_size']), method='N/r'):
  r_r_c = edf['rotated_r_c']

  all_avgs = np.array(r_r_c.groupby(level=0).transform('mean').to_list()).ravel()
  all_dists = np.abs(r_r_c - all_avgs)
  all_sigms = np.array(r_r_c.groupby(level=0).transform('std').to_list()).ravel()

  edf['mean_rotated_r_c'] = all_avgs
  edf['dist_from_mean_rotated_r_c'] = all_dists
  edf['rotated_r_c_sigm'] = all_sigms


  # Compute beta_step and r_step using NumPy functions
  beta_step = np.ptp(edf['beta'])
  r_step = np.ptp(edf['true_r_from_beta'])

  # Compute beta_intervals using NumPy linspace function
  num_of_groups = 5
  beta_intervals = np.linspace(edf['beta'].min(), edf['beta'].max(), num=num_of_groups)

  # Compute beta_group_to_bdf and true_r_group using NumPy operations
  beta_group_to_bdf = np.floor((num_of_groups * edf['beta'] + edf['beta'].max() - (num_of_groups + 1) * edf['beta'].min()) / beta_step).values
  true_r_group = np.floor((num_of_groups * edf['true_r_from_beta'] + edf['true_r_from_beta'].max() - (num_of_groups + 1) * edf['true_r_from_beta'].min()) / r_step).values

  bdf['beta_group'] = pd.Series(beta_group_to_bdf)
  bdf['true_r_group'] = pd.Series(true_r_group)

  signal_counts = edf['signal'].groupby(level=0).sum()
  bdf['signal_counts'] = signal_counts.values
  edf['signal_counts'] = np.take(signal_counts.values, edf.index.get_level_values(1))

  edf_to_bdf(edf.theta_p, bdf)
  bdf['cos_theta_p'] = np.cos(bdf['theta_p'])
  # edf_to_bdf(edf.signal_counts, bdf)
  edf_to_bdf(edf.beta, bdf)
  edf_to_bdf(edf.true_r_from_beta, bdf)
  edf_to_bdf(edf.corrected_true_r_from_beta, bdf)

  step = step / 2

  r_slices = np.arange(0, 800, step=step)

  def local_weighed_sum(r_slices, counts, max_index, n, method='N/r'):
    if method == 'N/r':
      sum_arr = r_slices[max_index - n:max_index + n] ** 2 * counts[max_index - n:max_index + n]
      den_arr = r_slices[max_index - n:max_index + n] * counts[max_index - n:max_index + n]
    else:
      sum_arr = r_slices[max_index - n:max_index + n] * counts[max_index - n:max_index + n]
      den_arr = counts[max_index - n:max_index + n]

    weighted_sum = np.sum(sum_arr)
    weighted_den = np.sum(den_arr)

    return weighted_sum / weighted_den


  def r_from_correction(x, n=1.0454, l=200, dl=17.5):
    return x * (1 + n * l / np.sqrt(dl ** 2 - x ** 2 * (1 - n ** 2)))

  def r_from_beta_and_correction(beta, n=1.0454, l=200, dl=17.5, n_max=1.05):
    return dl * np.tan(np.arccos(1 / beta / n)) + l * np.tan(np.arcsin(n_max * np.sqrt(1 - 1 / (beta * n ) ** 2)))

  r_correction_from_r = inversefunc(r_from_correction)
  r_interval = np.linspace(1, 100, num=200)
  r_correction_on_interval = r_correction_from_r(r_interval)
  # beta_from_r = inversefunc(r_from_beta_and_correction, domain=[0.95, 1])
  # beta_on_interval = beta_from_r(r_interval)

  true_r_from_beta = np.array(edf['true_r_from_beta'])
  # edf['corrected_true_r_from_beta'] = edf['true_r_from_beta'] + r_correction_on_interval[(true_r_from_beta / (r_interval[1] - r_interval[0])).astype(int)]
  edf['corrected_true_r_from_beta'] = r_from_beta_and_correction(edf['beta'])

  edf['slice_group'] = np.floor(r_r_c / step) * step
  for n_sigms in range(4, 5):
    all_counts_to_edf = []
    all_calculated_r = []
    all_calculated_r_fixed = []
    for entry, subentry in edf['slice_group'].groupby(level=0):
      counts = np.zeros(r_slices.shape)

      for subsubenrty in subentry:
        if 80 >= subsubenrty >= 16:  # Тут было ограничение сверху 70, снизу 10
          counts[int(np.round(subsubenrty / step, 0))] += 1 # а тут floor а не round

      if method == 'N/r':
        counts = np.array([(counts[i - 1] + counts[i]) / r_slices[i] for i in range(1, counts.shape[0])])
        # counts = np.array([(counts[i]) / r_slices[i] for i in range(1, counts.shape[0])])

      # max_index = counts.index(max(counts))
      max_index = np.where(counts == max(counts))[0][0]


      avg_r_from_slices = local_weighed_sum(r_slices, counts, max_index, n_sigms, method)
      # counts_to_edf = []
      for subsubenrty in subentry:
        all_counts_to_edf.append(counts[int(np.floor(subsubenrty / step))])
        all_calculated_r.append(avg_r_from_slices)
        if np.isnan(avg_r_from_slices):
          all_calculated_r_fixed.append(avg_r_from_slices  * (1 - 0.0775))
        else:
          all_calculated_r_fixed.append(avg_r_from_slices - r_correction_on_interval[int(avg_r_from_slices / (r_interval[1] - r_interval[0]))])


    edf[f'slice_counts_{n_sigms}_sigms'] = all_counts_to_edf

    edf[f'unfixed_calculated_r_{n_sigms}_sigms'] = all_calculated_r
    edf[f'calculated_r_{n_sigms}_sigms'] = all_calculated_r_fixed
    edf[f'delta_calc_true_r_{n_sigms}_sigms'] = edf[f'calculated_r_{n_sigms}_sigms'] - edf['true_r_from_beta']
    edf[f'eps_calc_true_r_{n_sigms}_sigms'] = edf[f'delta_calc_true_r_{n_sigms}_sigms'] / edf['true_r_from_beta'] * 100

    edf[f'beta_from_calc_r_{n_sigms}_sigms'] = np.sqrt((edf[f'calculated_r_{n_sigms}_sigms']) ** 2 + ((float(idf['distance']) + float(idf['W']) / 2) ** 2)) / (float(idf['n_mean']) * (float(idf['distance']))+float(idf['W'])/2)
    # edf[f'beta_from_calc_r_{n_sigms}_sigms'] = beta_on_interval[int(all_calculated_r / (r_interval[1] - r_interval[0]))]

    edf[f'delta_beta_{n_sigms}_sigms'] = edf[f'beta_from_calc_r_{n_sigms}_sigms'] - edf['beta']
    edf[f'eps_beta_{n_sigms}_sigms'] = edf[f'delta_beta_{n_sigms}_sigms'] / edf['beta'] * 100


    edf_to_bdf(edf[f'beta_from_calc_r_{n_sigms}_sigms'], bdf)
    edf_to_bdf(edf[f'delta_beta_{n_sigms}_sigms'], bdf)
    edf_to_bdf(edf[f'eps_beta_{n_sigms}_sigms'], bdf)
    edf_to_bdf(edf[f'unfixed_calculated_r_{n_sigms}_sigms'], bdf)
    edf_to_bdf(edf[f'calculated_r_{n_sigms}_sigms'], bdf)
    edf_to_bdf(edf[f'delta_calc_true_r_{n_sigms}_sigms'], bdf)
    edf_to_bdf(edf[f'eps_calc_true_r_{n_sigms}_sigms'], bdf)

  # all_counts_to_edf = np.array(all_counts_to_edf)
  # edf['slice_counts'] = all_counts_to_edf
  # edf['calculated_r'] = all_calculated_r
  # edf['delta_calc_true_r'] = np.abs(edf['calculated_r'] - edf['true_r_from_beta'])
  # edf['eps_calc_true_r'] = edf['delta_calc_true_r'] / edf['calculated_r'] *

  # edf['beta_from_calc_r'] = np.sqrt((edf['calculated_r']) ** 2 + (edf['r_p_c']) ** 2) / (float(idf['n_mean']) * edf['r_p_c'])
  # edf['delta_beta'] = np.abs(edf['beta_from_calc_r'] - edf['beta'])
  # edf['eps_beta'] = edf['delta_beta'] / edf['beta'] * 100
# rSlidingWindow(edf, idf, bdf)

In [42]:
def betaGroups(bdf: pd.DataFrame, avg_sigmas: tuple, avg_t_sigmas: tuple, n = 5):
  beta_sigms = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), n), 0.)
  beta_epss = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), n), 0.)
  beta_sigms_sigms = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), n), 0.)

  for group in range(1, n + 1):
    data = bdf[bdf['beta_group'] == group]
    for i in range(np.ptp(avg_sigmas)):
      for j in range(np.ptp(avg_t_sigmas)):
        population_fourth_moment = np.mean(bdf[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms'] ** 4)
        sample_fourth_moment = np.mean(data[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms'] ** 4)
        # print(np.std(data[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms']))
        beta_sigms[i, j, group - 1] = np.std(data[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms'])
        beta_epss[i, j, group - 1] = np.std(data[f'eps_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms'])
        beta_sigms_sigms[i, j, group - 1] = np.sqrt(2 * np.abs(sample_fourth_moment - population_fourth_moment) / (data.shape[0]))

  return beta_sigms, beta_epss, beta_sigms_sigms

In [43]:
def rms90(arr):
    # Calculate the mean and standard deviation of the array
    arr = arr.dropna()
    arr_mean = np.mean(arr)
    arr_std = np.std(arr)

    # Define the upper and lower limits for the 90% range
    lower_limit = np.percentile(arr, 5)
    upper_limit = np.percentile(arr, 95)
    # print(lower_limit, upper_limit)
    # print(arr)
    # Select the values within the 90% range
    arr_filtered = arr[(arr >= lower_limit) & (arr <= upper_limit)]
    # print(arr_filtered)
    assert arr_filtered.shape
    # Calculate the root mean square of the filtered values
    rms = np.std(arr_filtered)

    return rms

In [44]:
def betaGroupsRMS90(bdf: pd.DataFrame, avg_sigmas: tuple, avg_t_sigmas: tuple, n = 5):
  beta_sigms = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), n), 0.)
  beta_epss = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), n), 0.)
  beta_sigms_sigms = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), n), 0.)

  for group in range(1, n + 1):
    data = bdf[bdf['beta_group'] == group]
    for i in range(np.ptp(avg_sigmas)):
      for j in range(np.ptp(avg_t_sigmas)):
        population_fourth_moment = np.mean(bdf[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms'] ** 4)
        sample_fourth_moment = np.mean(data[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms'] ** 4)
        # print(np.std(data[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms']))
        beta_sigms[i, j, group - 1] = rms90(data[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms'])
        # assert not np.isnan(beta_sigms[i, j, group - 1])
        beta_epss[i, j, group - 1] = rms90(data[f'eps_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms'])
        beta_sigms_sigms[i, j, group - 1] = np.sqrt(2 * np.abs(sample_fourth_moment - population_fourth_moment) / (data.shape[0]))

  return beta_sigms, beta_epss, beta_sigms_sigms

In [45]:
def betaGroupsGauss(edf: pd.DataFrame, bdf: pd.DataFrame, avg_sigmas: tuple, avg_t_sigmas: tuple, n = 5):
  def gaussian(x, mean, sigma):
    return norm.pdf(x, loc=mean, scale=sigma)
  beta_sigms = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), n), 0.)
  beta_sigms_errs = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), n), 0.)

  for group in range(1, n + 1):
    data = bdf[bdf['beta_group'] == group]
    data = data.dropna()
    for i in range(np.ptp(avg_sigmas)):
      for j in range(np.ptp(avg_t_sigmas)):
        # print(np.std(data[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms']))
        bin_heights, bin_borders = np.histogram(data[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms'], bins='doane', normed=True)
        bin_width = np.diff(bin_borders)
        bin_centers = bin_borders[:-1] + bin_width / 2
        p0 = norm.fit(data[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms'])
        try:
          gauss_param, cov = curve_fit(gaussian, bin_centers, bin_heights, maxfev=50000, p0=norm.fit(data[f'delta_beta_{i + avg_sigmas[0]}_rsigms_{j + avg_t_sigmas[0]}_tsigms']), method='trf')
        except ValueError:
          gauss_param, cov = curve_fit(gaussian, bin_centers, bin_heights, maxfev=50000, method='trf')
        cov = np.sqrt(np.diag(cov))
        beta_sigms[i, j, group - 1] = gauss_param[1]
        beta_sigms_errs[i, j, group - 1] = cov[1]

  return beta_sigms, beta_sigms_errs

In [46]:
# labels = ['0', '1e3', '1e4', '1e5', '1e6']
# labels = ['DCR = ' + i + ' $Hz/mm^2$' for i in labels]
# num_of_groups = 10
# y = np.arange(1, num_of_groups + 1)
# x = (y * (max(edf['beta']) - min(edf['beta'])) - max(edf['beta']) + (num_of_groups + 1) * min(edf['beta'])) / num_of_groups
# plt.figure(figsize=(10, 10))
# for i in range(len(labels)):
#   plt.plot(x, [0.0005] * 10, label=labels[i])
# plt.ylim((0, 0.002))
# # plt.xlim((0.954, 0.999))
# plt.legend()
# plt.grid()
# plt.show()

In [47]:
def plot_final_graph(beta_sigms, beta_sigms_yerr, avg_sigmas, avg_t_sigmas, r_width, t_width, r_factor, t_factor, weighed, to_save=True, deg_lim=False, num_of_groups=10):
  labels = ['0', '1e3', '1e4', '1e5', '1e6']
  labels = ['DCR = ' + i + ' $Hz/mm^2$' for i in labels]
  colors = ['c', 'y', 'g', 'r', 'm']
  weight = 'weighed' if weighed else 'unweighed'
  y = np.arange(1, num_of_groups + 1)
  x = (y * (max(edf['beta']) - min(edf['beta'])) - max(edf['beta']) + (num_of_groups + 1) * min(edf['beta'])) / num_of_groups

  fig, axs = plt.subplots(np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), figsize=(10 * np.ptp(avg_t_sigmas), 10 * np.ptp(avg_sigmas)))
  title = f'Method: N(r) / r; {weight} Avg\nR Width = {r_width}mm, T Width = {t_width}ns\nR step factor = {r_factor}, T step factor = {t_factor}'
  if deg_lim:
    title += '\n' + r'$\theta_p < 10\deg$'
  # fig.suptitle(title)

  if np.ptp(avg_sigmas) > 1:
    for i in range(np.ptp(avg_sigmas)):
      for j in range(np.ptp(avg_t_sigmas)):
        for k in range(beta_sigms.shape[0]):
          axs[i, j].plot(x, beta_sigms[k, i, j], label=labels[k], c=colors[k])
          axs[i, j].errorbar(x, beta_sigms[k, i, j], xerr=[np.diff(x)[0]/4 for _ in x], linestyle='', c=colors[k])
          axs[i, j].errorbar(x, beta_sigms[k, i, j], yerr=beta_sigms_yerr[k, i, j], linestyle='', c=colors[k])
        axs[i, j].legend(loc='upper right')
        axs[i, j].set_xlabel('Beta Group')
        axs[i, j].set_ylabel(r'RMS90($\Delta\beta$)')
        if deg_lim:
          axs[i, j].set_ylim((0, 0.004))
        axs[i, j].set_title(f'Velocity resoultion for\nr window width = {avg_sigmas[0] + i}$\sigma$\nt window width = {avg_t_sigmas[0] + j}$\sigma$')
        axs[i, j].grid()
  elif np.ptp(avg_t_sigmas) > 1:
    for j in range(np.ptp(avg_t_sigmas)):
      for k in range(beta_sigms.shape[0]):
        axs[j].plot(x, beta_sigms[k, 0, j], label=labels[k], c=colors[k])
        axs[j].errorbar(x, beta_sigms[k, 0, j], xerr=[np.diff(x)[0]/4 for _ in x], linestyle='', c=colors[k])
        axs[j].errorbar(x, beta_sigms[k, 0, j], yerr=beta_sigms_yerr[k, 0, j], linestyle='', c=colors[k])
      axs[j].legend(loc='upper right')
      axs[j].set_xlabel('Beta Group')
      axs[j].set_ylabel(r'RMS90($\Delta\beta)$')
      if deg_lim:
        axs[j].set_ylim((0, 0.004))
      axs[j].set_title(f'Velocity resoultion for\nr window width = {avg_sigmas[0]}$\sigma$\nt window width = {avg_t_sigmas[0] + j}$\sigma$')
      axs[j].grid()
  else:
    for k in range(beta_sigms.shape[0]):
      axs.plot(x, beta_sigms[k, 0, 0], label=labels[k], c=colors[k])
      axs.errorbar(x, beta_sigms[k, 0, 0], xerr=[np.diff(x)[0]/4 for _ in x], linestyle='', c=colors[k])
      axs.errorbar(x, beta_sigms[k, 0, 0], yerr=beta_sigms_yerr[k, 0, 0], linestyle='', c=colors[k])
    axs.legend(loc='upper right')
    axs.set_xlabel('Beta Group')
    axs.set_ylabel(r'RMS90($\Delta\beta$)')
    if deg_lim:
      axs.set_ylim((0, 0.002))
    axs.set_title(f'Velocity resoultion for\nr window width = {avg_sigmas[0]}$\sigma$\nt window width = {avg_t_sigmas[0]}$\sigma$')
    axs.grid()

  if to_save:
    filename = f'{weight}_avg_rw={r_width}_tw={t_width}_rs={r_factor}_ts={t_factor}_rsigms={avg_sigmas[0]}-{avg_sigmas[-1]-1}_tsigms={avg_t_sigmas[0]}-{avg_t_sigmas[-1]-1}'
    if deg_lim:
      filename += '_10deg'
    filename += '.png'
    fig.savefig(os.path.join('results', f'{filename}'))
    plt.close(fig)
  else:
    plt.show()

In [48]:
def plot_groupped_distributions(bdf, avg_sigmas, avg_t_sigmas, r_width, t_width, r_factor, t_factor, weighed, background, to_save=True, deg_lim=False, num_of_groups=10):
  labels = ['0', '1e3', '1e4', '1e5', '1e6']
  labels = ['DCR = ' + i + ' cps' for i in labels]
  colors = ['c', 'y', 'g', 'r', 'm']
  weight = 'weighed' if weighed else 'unweighed'
  left_lim = min(bdf[f'delta_beta_{avg_sigmas[0]}_rsigms_{avg_t_sigmas[0]}_tsigms'])
  right_lim = max(bdf[f'delta_beta_{avg_sigmas[0]}_rsigms_{avg_t_sigmas[0]}_tsigms'])
  y = np.arange(1, num_of_groups + 1)
  x = (y * (max(edf['beta']) - min(edf['beta'])) - max(edf['beta']) + (num_of_groups + 1) * min(edf['beta'])) / num_of_groups
  x_interval_over_2 = (x[1] - x[0]) / 2
  fig, axs = plt.subplots(num_of_groups, 1, figsize=(16, 9 * num_of_groups), sharex=True)
  title = f'Method: N(r) / r; {weight} Avg\nR Width = {r_width}mm, T Width = {t_width}ns\nR step factor = {r_factor}, T step factor = {t_factor}, DCR={background}'
  if deg_lim:
    title += '\n' + r'$\theta_p < 10\deg$'
  # fig.suptitle(title)
  fig.tight_layout()
  if np.ptp(avg_sigmas) > 1:
    pass
  elif np.ptp(avg_t_sigmas) > 1:
    pass
  else:
    for group in range(1, num_of_groups + 1):
      data = bdf[bdf['beta_group'] == group]
      # data = data.dropna()
      # print(data.shape)
      # bin_heights, bin_borders = np.histogram(data[f'delta_beta_{avg_sigmas[0]}_rsigms_{avg_t_sigmas[0]}_tsigms'], bins='auto', normed=True)
      # bin_width = np.diff(bin_borders)
      axs[group - 1].hist(data[f'delta_beta_{avg_sigmas[0]}_rsigms_{avg_t_sigmas[0]}_tsigms'], bins='auto')
      # axs[group - 1].set_xlabel(r'$\Delta\beta$')
      axs[group - 1].set_ylabel(r'Events')
      axs[group - 1].set_title(r'$\beta$ in ' + f'[{round(x[group - 1] - x_interval_over_2, 3)}, {round(x[group -1 ] + x_interval_over_2, 3)}]')
      axs[group - 1].set_xlim((-0.05, 0.08))

  if to_save:
    filename = f'Hist_{weight}_avg_rw={r_width}_tw={t_width}_rs={r_factor}_ts={t_factor}_rsigms={avg_sigmas[0]}-{avg_sigmas[-1]-1}_tsigms={avg_t_sigmas[0]}-{avg_t_sigmas[-1]-1}_background={background}'
    if deg_lim:
      filename += '_10deg'
    filename += '.png'
    fig.savefig(os.path.join('results', f'{filename}'))
    plt.close(fig)
  else:
    plt.show()

In [49]:
# tmp1, tmp2, tmp3 = betaGroups(edf_d, bdf_d, avg_sigmas=avg_sigmas, avg_t_sigmas=avg_t_sigmas, n=10)

In [50]:
# tmp2.shape

In [51]:
# plt.plot(x, tmp1[0, j], label=labels[4], c=colors[4])
# # plt.errorbar(x, tmp1[0, j], xerr=[np.diff(x)[0]/4 for _ in x], linestyle='', c=colors[4])
# plt.errorbar(x, tmp1[0, j], yerr=tmp3[0, j], linestyle='', c=colors[4])
#
# plt.grid()
# plt.show()

In [52]:
# for index, meas_r in bdf['calculated_r_4_sigms'].groupby(level=0):
#   # print(meas_r)
#   break
# print(bdf.loc[0].beta)

In [53]:
# print([[[]]* 5] * 5)
# print(cal_arr.shape) # r_sigms -> t_sigms -> thetq_groups -> 3 params


In [54]:
# edf.hist('time_slice_group')

In [55]:
# edf[np.isfinite(edf.unfixed_calculated_r_2d_1_rsigms_1_tsigms)].hist('time_slice_group')

In [56]:
def pol2old(x, b, c, d):
  return b * x ** 2 + c * x + d

In [57]:
def pol2(x, p0, p1, p2):
  return p0 + p1 * x + p2 * x ** 2

In [58]:
def d3pol2(X, p0, p1, p2, q0, q1, q2, k0, k1, k2):
  r, theta = X
  # return pol(r, pol2(theta, p0, p1, p2), pol2(theta, q0, q1, q2), pol2(theta, k0, k1, k2))
  return pol(r, p0 + p1 * theta + p2 * theta ** 2, q0 + q1 * theta + q2 * theta ** 2, k0 + k1 * theta + k2 * theta ** 2)

In [59]:
def calibration(edf: pd.DataFrame, bdf: pd.DataFrame, avg_sigmas: tuple, avg_t_sigmas: tuple, step=float(idf.pixel_size), method='N/r', t_window_width=2,
                        r_width_factor=2, t_width_factor=8, full_width_t_hist = False, weighed=True, deg_lim=False, param_fit=False):
  def gaussian(x, mean, sigma):
    return norm.pdf(x, loc=mean, scale=sigma)
  # to_return_unbinned = [[], [], [], []]
  bdf['cos_theta_p'] = np.cos(bdf['theta_p'])
  theta_p_max = max(bdf.cos_theta_p)
  theta_p_min = min(bdf.cos_theta_p)
  num_of_theta_intervals = 11
  theta_intervals = np.linspace(theta_p_min, theta_p_max, num=num_of_theta_intervals)
  theta_dif = (theta_intervals[1:] + theta_intervals[:-1]) / 2
  to_return_unbinned = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), num_of_theta_intervals - 1, 3), 0.)

  errs_tmp = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), num_of_theta_intervals - 1, 3), 0.)
  fit_params = np.full((np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), 3, 3), 0.)

  weight = 'weighed' if weighed else 'unweighed'
  beta_min = min(bdf.beta)
  num_of_beta_intervals = 10 # was 20
  beta_delim = (max(bdf.beta) - min(bdf.beta)) / num_of_beta_intervals
  beta_intervals = np.linspace(min(bdf.beta), max(bdf.beta), num=num_of_beta_intervals + 1)
  dir_to_save = f'{weight}_rw={step}_tw={t_window_width}_rs={r_width_factor}_ts={t_width_factor}'
  if not os.path.exists(os.path.join('calibrations', dir_to_save)):
    os.mkdir(os.path.join('calibrations', dir_to_save))
  for r_sigms in range(*avg_sigmas):
    fig, axs = plt.subplots(num_of_theta_intervals - 1, np.ptp(avg_t_sigmas), figsize=(16 * np.ptp(avg_t_sigmas), 90))
    for t_sigms in range(*avg_t_sigmas):
      chosen_column = f'unfixed_calculated_r_2d_{r_sigms}_rsigms_{t_sigms}_tsigms'

      meas_r_min = min(bdf[chosen_column])
      meas_r_max = max(bdf[chosen_column])
      num_of_meas_r_intervals = num_of_beta_intervals
      meas_r_delim = (meas_r_max - meas_r_min) / num_of_meas_r_intervals
      meas_r_intervals = np.linspace(meas_r_min, meas_r_max, num=num_of_meas_r_intervals + 1)


      # fig2, axs2 = plt.subplots(10* num_of_theta_intervals - 10, 10, figsize=(16, 90))

      for theta_interval_index in range(num_of_theta_intervals - 1):
        xerrs = []
        yerrs = []
        gauss_beta = []
        gauss_r = []
        t_bdf = bdf.copy()
        t_bdf = t_bdf[np.isfinite(t_bdf[chosen_column])]
        t_bdf = t_bdf[t_bdf.signal_counts >= 15]
        t_bdf = t_bdf[t_bdf.cos_theta_p <= theta_intervals[theta_interval_index + 1]]
        t_bdf = t_bdf[t_bdf.cos_theta_p >= theta_intervals[theta_interval_index]]
        if t_sigms - avg_t_sigmas[0] != 0:
          for_colorbar = axs[theta_interval_index, t_sigms - avg_t_sigmas[0]].hist2d(t_bdf[chosen_column], t_bdf.beta, bins=70, range=((0, 80), (beta_min,1)))
          fig.colorbar(for_colorbar[3], ax=axs[theta_interval_index, t_sigms - avg_t_sigmas[0]])
        else:
          for_colorbar = axs[theta_interval_index].hist2d(t_bdf[chosen_column], t_bdf.beta, bins=70, range=((0, 80), (beta_min,1)))
          fig.colorbar(for_colorbar[3], ax=axs[theta_interval_index])
        t_bdf = t_bdf[t_bdf[chosen_column] <= 65]
        t_bdf = t_bdf[t_bdf[chosen_column] >= 25]
        # meas_r_in_beta_interval = []
        # beta_in_meas_r_interval = []
        # for _ in range(num_of_beta_intervals + 1):
        #   meas_r_in_beta_interval.append([])
        # for _ in range(num_of_meas_r_intervals + 1):
        #   beta_in_meas_r_interval.append([])
        # for index, meas_r in t_bdf[chosen_column].groupby(level=0):
        #   meas_r_in_beta_interval[int(round((t_bdf.loc[index].beta - beta_min) / beta_delim, 0))].append(meas_r.values[0])
        #   beta_in_meas_r_interval[int(round((meas_r - meas_r_min) / meas_r_delim, 0))].append(t_bdf.loc[index].beta)

        pol_param, cov = curve_fit(pol, t_bdf[chosen_column], t_bdf.beta, maxfev=50000, p0=(1, 80, 30))
        # print(pol_param)
        to_return_unbinned[r_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][theta_interval_index] = pol_param
        # to_return_unbinned[sigms].append(pol_param)
        pol_param_errs = np.sqrt(np.diag(cov))
        errs_tmp[r_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][theta_interval_index] = pol_param_errs
        rs = np.linspace(10, 80, num=50)
        chi2 = np.sum((t_bdf.beta - pol(t_bdf[chosen_column], *pol_param)) ** 2)
        #

        # axs[theta_interval_index].plot(rs, pol(rs, *(pol_param + pol_param_errs)), c='y')
        # axs[theta_interval_index].plot(rs, pol(rs, *(pol_param - pol_param_errs)), c='g')
        # axs[theta_interval_index].legend()
        #
        # ind_to_remove = []
        # for meas_r_index, meas_rs in enumerate(meas_r_in_beta_interval):
        #   if len(meas_rs) > 1:
        #     # bin_heights, bin_borders = np.histogram(meas_rs, bins='doane', normed=True)
        #     # bin_width = np.diff(bin_borders)
        #     # bin_centers = bin_borders[:-1] + bin_width / 2
        #     # p0 = norm.fit(meas_rs)
        #     # try:
        #     #   gauss_param, cov = curve_fit(gaussian, bin_centers, bin_heights, maxfev=50000, p0=norm.fit(meas_rs), method='trf')
        #     # except ValueError:
        #     #   gauss_param, cov = curve_fit(gaussian, bin_centers, bin_heights, maxfev=50000, method='trf')
        #     # cov = np.sqrt(np.diag(cov))
        #     # gauss_r.append(np.mean(meas_rs))
        #     xerrs.append(0)
        #   else:
        #     xerrs.append(0)
        #     # gauss_r.append(0)
        #     ind_to_remove.append(meas_r_index)
        #
        # for beta_index, betas in enumerate(beta_in_meas_r_interval):
        #   if len(betas) > 1:
        #     betas = np.array(betas)
        #     bin_heights, bin_borders = np.histogram(betas, bins='doane', normed=True)
        #     bin_width = np.diff(bin_borders)
        #     bin_centers = bin_borders[:-1] + bin_width / 2
        #     max_height_ind = np.argmax(bin_heights)
        #     beta_peak = bin_centers[max_height_ind]
        #     std = np.std(betas)
        #     betas = betas[(betas >= beta_peak - std) & (betas <= beta_peak + std)]
        #     bin_heights, bin_borders = np.histogram(betas, bins='doane', normed=True)
        #     bin_width = np.diff(bin_borders)
        #     bin_centers = bin_borders[:-1] + bin_width / 2
        #     p0 = norm.fit(betas)
        #     gauss_param, cov = curve_fit(gaussian, bin_centers, bin_heights, maxfev=50000, p0=norm.fit(betas), method='trf')
        #     cov = np.sqrt(np.diag(cov))
        #     yerrs.append(gauss_param[1])
        #     if not np.isfinite(cov[0]):
        #       ind_to_remove.append(beta_index)
        #     gauss_beta.append(gauss_param[0])
        #     gauss_r.append(meas_r_intervals[beta_index])
        #     # axs2[((theta_interval_index) * 10 + beta_index) // 10, beta_index % 10].hist(betas, bins='doane')
        #   else:
        #     gauss_beta.append(0.95)
        #     gauss_r.append(0)
        #     yerrs.append(0)
        #     ind_to_remove.append(beta_index)
        # mask = list(set(map(int, np.linspace(0, num_of_beta_intervals - 1, num=num_of_beta_intervals))).difference(set(ind_to_remove)))
        # # print(ind_to_remove)
        # gauss_r = np.array(gauss_r)
        # gauss_beta = np.array(gauss_beta)
        # xerrs = np.array(xerrs)
        # yerrs = np.array(yerrs)
        #
        # meas_r_to_plot = []
        # beta_to_plot = []
        # gauss_beta_to_2d = []
        # for i in range(num_of_beta_intervals + 1):
        #   for j in range(len(meas_r_in_beta_interval[i])):
        #     meas_r_to_plot.append(meas_r_in_beta_interval[i][j])
        #     beta_to_plot.append(beta_intervals[i])
        #     gauss_beta_to_2d.append(gauss_beta[i])

        # gauss_r = gauss_r[mask]
        # gauss_beta = gauss_beta[mask]
        # xerrs = xerrs[mask]
        # yerrs = yerrs[mask]
        # pol_param_from_gauss, pol_cov_from_gauss = curve_fit(pol, gauss_r, gauss_beta, sigma=yerrs, absolute_sigma=True, maxfev=50000)
        # to_return_binned.append(pol_param_from_gauss)
        # pol_cov_from_gauss = np.sqrt(np.diag(pol_cov_from_gauss))
        # print(pol_param_from_gauss)

        # axs[theta_interval_index].plot(rs, pol(rs, *pol_param_from_gauss), c='r')
        # axs[theta_interval_index].plot(rs, pol(rs, *(pol_param_from_gauss + pol_cov_from_gauss)), c='y')
        # axs[theta_interval_index].plot(rs, pol(rs, *(pol_param_from_gauss - pol_cov_from_gauss)), c='g')

        # axs[theta_interval_index].hist2d(t_bdf[chosen_column], t_bdf.beta, bins=70, range=((0, 80), (beta_min,1)))
        #
        # axs[theta_interval_index].scatter(t_bdf[chosen_column], t_bdf.beta)

        # axs[theta_interval_index].hist2d(meas_r_to_plot, gauss_beta_to_2d, bins=50, range=((0, 70), (min(t_bdf.beta), max(t_bdf.beta))))
        # axs[theta_interval_index].scatter(meas_r_to_plot, gauss_beta_to_2d)

        # axs[theta_interval_index].scatter(meas_r_to_plot, beta_to_plot)

        # axs[theta_interval_index].scatter(gauss_r, gauss_beta)
        # axs[theta_interval_index].errorbar(gauss_r, gauss_beta, xerr=xerrs, linestyle='', c='r')
        # axs[theta_interval_index].errorbar(gauss_r, gauss_beta, yerr=yerrs, linestyle='', c='g')


        # axs[theta_interval_index].scatter([np.mean(r) for r in meas_r_in_beta_interval], gauss_beta)
        # axs[theta_interval_index].errorbar([np.mean(r) for r in meas_r_in_beta_interval], gauss_beta, xerr=xerrs, linestyle='', c='r')
        # axs[theta_interval_index].errorbar([np.mean(r) for r in meas_r_in_beta_interval], gauss_beta, yerr=yerrs, linestyle='', c='g')
        if t_sigms - avg_t_sigmas[0] != 0:
          axs[theta_interval_index, t_sigms - avg_t_sigmas[0]].plot(rs, pol(rs, *pol_param), label=r'$\chi^2$ = '+ str(chi2), c='r')
          axs[theta_interval_index, t_sigms - avg_t_sigmas[0]].set_xlim((0, 90))
          axs[theta_interval_index, t_sigms - avg_t_sigmas[0]].set_ylim((0.955, 1))
          axs[theta_interval_index, t_sigms - avg_t_sigmas[0]].set_ylim((beta_min, 1))
          axs[theta_interval_index, t_sigms - avg_t_sigmas[0]].set_xlabel(r'$R_{reco}$, mm')
          axs[theta_interval_index, t_sigms - avg_t_sigmas[0]].set_ylabel(r'$\beta_{true}$')
        else:
          axs[theta_interval_index].plot(rs, pol(rs, *pol_param), label=r'$\chi^2$ = '+ str(chi2), c='r')
          axs[theta_interval_index].set_xlim((0, 90))
          axs[theta_interval_index].set_ylim((0.955, 1))
          axs[theta_interval_index].set_ylim((beta_min, 1))
          axs[theta_interval_index].set_xlabel(r'$R_{reco}$, mm')
          axs[theta_interval_index].set_ylabel(r'$\beta_{true}$')
      if param_fit:
        t_bdf = bdf.copy()
        t_bdf = t_bdf[np.isfinite(t_bdf[chosen_column])]
        t_bdf = t_bdf[t_bdf.signal_counts >= 5]
        X = (np.array(t_bdf[chosen_column]), np.array(t_bdf.cos_theta_p))
        fit, errs = curve_fit(d3pol2, X, t_bdf.beta, p0=(1.219, -0.5588, 0.2946, 864.4, -1922, 1055, -2535, 6572, -3751))
        errs = np.sqrt(np.diag(errs))
        for param in range(3):
          fit_params[r_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][param] = fit[param * 3: param * 3 + 3]
        print(fit)
        print(errs)
        chi2 = np.sum((t_bdf.beta - d3pol2(X, *fit)) ** 2)
        print(chi2)
      # if param_fit:
      #   for param in range(3):
      #     fit, _ = curve_fit(pol2, theta_dif, to_return_unbinned[r_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][:, param], sigma=errs_tmp[r_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][:, param])
      #     fit_params[r_sigms - avg_sigmas[0]][t_sigms - avg_t_sigmas[0]][param] = fit
    filename = f'rsigm={r_sigms}_t_sigms={avg_t_sigmas[0]}-{avg_t_sigmas[-1] - 1}'
    if deg_lim:
      filename += '_10deg'
    if dir_to_save != '':
      fig.savefig(os.path.join('calibrations', dir_to_save, f'{filename}'))
      plt.close(fig)

        # print(len(beta_to_plot))
    # plt.show()
  # if param_fit:
  #   return fit_params
  return to_return_unbinned
# cal_arr = np.array(calibration(edf, bdf, avg_sigmas=(1, 5), avg_t_sigmas=(1, 5))) # add r and t calibr - done

In [60]:
ы

NameError: name 'ы' is not defined

In [ ]:
fig, axs = plt.subplots(2)
h = axs[0].hist2d(edf.x_c, edf.y_c, bins=100)
h2 = axs[1].hist2d(edf.x_c, edf.y_c, bins=100)
fig.colorbar(h[3], ax=axs[0])
fig.colorbar(h2[3], ax=axs[1])

plt.show()

In [ ]:
r_slices = np.arange(0, 800, step=float(idf.pixel_size)/2)

In [ ]:
r_slices = r_slices[r_slices <= 80]

In [ ]:
r_slices

In [ ]:
bdf_tmp.unfixed_calculated_r_2d_1_rsigms_1_tsigms

In [ ]:
bdf.unfixed_calculated_r_2d_1_rsigms_1_tsigms

In [ ]:
edf_tmp = edf[edf.cos_theta_p <= .8]
bdf_tmp = bdf[bdf.cos_theta_p <= .8]

In [ ]:
plt.hist(bdf_tmp.unfixed_calculated_r_2d_1_rsigms_1_tsigms, bins=30)
plt.show()

In [ ]:
plt.hist2d(edf_tmp.unfixed_calculated_r_2d_1_rsigms_1_tsigms, edf_tmp.beta,bins=70, range=((0, 80), (0.955,1)))
plt.show()

In [ ]:
edf_tmp = edf[np.cos(edf.theta_p) <= 0.8]

In [ ]:
cal_arr.shape

In [ ]:
cal_arr

In [ ]:
# plt.hist(bdf.unfixed_calculated_r_1_sigms, bins=100)
# plt.show()

In [ ]:
# print(bdf[np.isfinite(bdf.unfixed_calculated_r_4_sigms)].shape)
# plt.hist(bdf.unfixed_calculated_r_4_sigms)
# plt.show()

In [ ]:
for i in range(2):
  print(i)

In [ ]:
signalLength(edf)

In [ ]:
recoAngles(edf, idf)

In [ ]:
edf = applySpaceCut(edf)

In [ ]:
planeRecalculation(edf, idf)

In [ ]:
planeRotation(edf)
# testRotation2(edf)

In [ ]:
edf = applySecondSpaceCut(edf)

In [ ]:
primaryDirectionRecalculation(edf)

In [ ]:
recoAngles(edf, idf, rotation_mode=True)

In [ ]:
# optimal_width = optimalWidth(edf)

In [ ]:
bdf = pd.DataFrame()

In [ ]:
# rSlidingWindowOld(edf, idf, bdf)

In [ ]:
# rSlidingWindowIntro(edf, idf, bdf, avg_sigmas=(1, 5), cal_arr=cal_arr, avg_t_sigmas=(1, 5))

In [ ]:
# calculateSignalCounts(edf, bdf)

In [ ]:
cal_arr = False

In [ ]:
cal_arr = rSlidingWindow(edf, idf, bdf, (1, 3), (1, 3), num_of_groups=10)

In [ ]:
# rSlidingWindowLoop1(edf, idf, bdf, avg_sigmas=(1, 5), avg_t_sigmas=(1, 5), cal_arr=cal_arr, full_width_t_hist=True)

In [ ]:
# rSlidingWindowLoop2(edf, idf, bdf,  avg_sigmas=(1, 5), avg_t_sigmas=(1, 5), cal_arr=cal_arr)

In [ ]:
# print(betaGroups(edf, bdf))

In [ ]:
#edf_with_noise = next(genChunkFromRoot(filepath, 10000, noisefreqpersqmm=1e5))
#recoAngles(edf_with_noise, idf)

In [ ]:
#(edf, idf)

In [ ]:
edf.shape

In [ ]:
bdf.shape

In [ ]:
tmp_edf = edf[edf.theta_p <= 10 * np.pi / 180]
tmp_bdf = bdf[bdf.theta_p <= 10 * np.pi / 180]

In [ ]:
tmp_edf

In [ ]:
edf

In [ ]:
bdf

In [ ]:
10000 / (bdf[bdf.theta_p <= 10. * np.pi / 180].shape[0] / bdf.shape[0])

In [ ]:
plt.hist(bdf[bdf.theta_p <= 10. * np.pi / 180].theta_p)
plt.show()

In [ ]:
# edf.loc[1444].to_csv('test.csv')

In [ ]:
# plt.hist(bdf.beta, bins='auto')
# plt.xlabel(r'$\beta$')
# plt.show()

In [ ]:
# plt.hist(bdf.signal_counts, bins='auto')
# plt.show()

In [ ]:
#edf_with_noise

In [ ]:
# for col in edf:
#   print(col)

In [ ]:
ы

In [ ]:
#edf['beta'].max()
#edf[edf['beta'] == edf['beta'].max()].index
# 4363 - max beta event

In [ ]:
#edf['beta'][4363]

In [ ]:
def plothits(edf: pd.DataFrame, events: int = 1, first: int = 0, trange: tuple = None, extended_mode = True, dir_to_save = ''):
  '''
  Отрисовка срабатываний в датафрейме edf в виде распределений по координатам, углам и времени.
  Параметры:
  edf - датафрейм для отрисовки
  events - число событий для отрисовки
  first - рисовать, начиная с этого события
  '''  
  
  edf = edf.loc[(first, 0):(first+events, -1)]
  # if events == 1:
    # rdf = rdf[rdf['index'] == first]

  xh = edf['x_c'].to_numpy()
  yh = edf['y_c'].to_numpy()
  zh = edf['z_c'].to_numpy()
  th = edf['t_c'].to_numpy()
  sh = edf['signal'].to_numpy()
  x_i = edf['x_i'].to_numpy()
  y_i = edf['y_i'].to_numpy()
  xh_r = edf['rotated_x'].to_numpy()
  yh_r = edf['rotated_y'].to_numpy()
  zh_r = edf['rotated_z'].to_numpy()
  x_i_r = edf['rotated_x_i'].to_numpy()
  y_i_r = edf['rotated_y_i'].to_numpy()
  toh = edf['t_c_orig'].to_numpy()
  theta_c = edf['theta_c'].to_numpy()
  phi_c = edf['phi_c'].to_numpy()
  theta_c_r = edf['rotated_theta_c'].to_numpy()
  phi_c_r = edf['rotated_phi_c'].to_numpy()
  r_c = edf['r_c'].to_numpy()
  theta_p = edf['theta_p'].to_numpy()
  nh = edf['nhits'].to_numpy()
  bh = edf['beta'].to_numpy()
  r_c_r = edf['rotated_r_c'].to_numpy()
  slices = edf['slice_group'].to_numpy()
  counts = edf['slice_counts'].to_numpy()
  true_beta = edf['beta'].to_numpy()
  # calc_beta = edf['beta_from_calc_r'].to_numpy()
  # delta_beta = edf['delta_beta'].to_numpy()
  # eps_beta = edf['eps_beta'].to_numpy()
  true_r = edf['true_r_from_beta'].to_numpy()
  # calc_r = edf['calculated_r'].to_numpy()
  # delta_r = edf['delta_calc_true_r'].to_numpy()
  # eps_r = edf['eps_calc_true_r'].to_numpy()

  # if events == 1:
    # slices = rdf['slice'].to_numpy()
    # counts = rdf['counts'].to_numpy()
  #print(cth)
  #print(f'Number of events to plot: {events}')
  #print('Momenta, MeV/c:', ' '.join([f'{p:.1f}' for p in edf['momentum'].unique()]))
  #print('Incident angles, degrees:', ' '.join([f'{t:.1f}' for t in edf['theta_p'].unique()*180/np.pi]))
  #print(f"Number of hits to plot: {sh.size}")
  #print(f"Number of signal hits: {sh.sum():d}")
  #print("(Xmin, Xmax):", xh.min(), xh.max())
  #print("(Tmin, Tmax):", th.min(), th.max())


  def plot_extra_graph(eh, label, units, to_save = True, raw = False, close = True, pic_name=''):
    fige = plt.figure(figsize=(6,6))
    try:
      if not sh.all():
        _, bins, _ = plt.hist([eh[sh], eh[~sh]], bins=100, color=['r', 'k'], histtype='stepfilled', label=['Signal', 'Noise'])
        plt.legend()
      else:
        _, bins, _ = plt.hist(eh[sh], bins=100, range=trange, color='r', histtype='stepfilled')
      de = bins[1] - bins[0]
      plt.xlabel(f'{label}, {units}', fontsize=14)
      plt.ylabel(f'Hits/({de:.3f} {units})', fontsize=14)
      if (isinstance(trange, tuple) or isinstance(trange, list)) and len(trange) == 2:
        plt.xlim(trange)
      plt.tight_layout()
      if not pic_name:
        pic_name = label
      if dir_to_save != '' and to_save:
        if raw:
          fige.savefig(os.path.join(picsdir, dir_to_save, 'raw', f'{pic_name}.png'))
        else:
          fige.savefig(os.path.join(picsdir, dir_to_save, f'{pic_name}.png'))
    except ValueError:
      pass
    if close:
      plt.close(fige)


  def plotgraph(x, y, xlim = 0, ylim = 0, xlabel = 'NONE', ylabel = 'NONE', plot_intersection = False, ix = 0, iy = 0, filename='', close = True, ignore_sh = False, figsize=(6,6)):
    fig = plt.figure(figsize=figsize)
    if ignore_sh:
      plt.plot(x, y, 'r.', label='Data')
    else:
      if x is not None and y is not None:
        plt.plot(x[sh], y[sh], 'r.', label='Signal')  # сигнальные срабатывания
        if not sh.all():
          plt.plot(x[~sh], y[~sh], 'k.', label='Noise')  # шумовые срабатывания
          plt.legend()
      if plot_intersection:
        plt.plot(ix, iy, 'g.', label='Intersection')
        plt.legend()
    #plt.plot([-400, 400], [0, 0], 'k-')
    #plt.plot([0, 0], [-400, 400], 'k-')
    if xlim != 0:
      plt.xlim(xlim[0], xlim[1])
    if ylim != 0:
      plt.ylim(ylim[0], ylim[1])

    plt.xlabel(xlabel, fontsize=14)
    plt.ylabel(ylabel, fontsize=14)
    plt.tight_layout()
    if dir_to_save != '':
      fig.savefig(os.path.join(picsdir, dir_to_save, f'{filename}'))
    if close:
      plt.close(fig)


  if events == 1:
    plotgraph(xh_r, yh_r, (-415, 415), (-415, 415), 'XHit_rotated, mm', 'YHit_rotated, mm', True, x_i_r, y_i_r, filename='XY_rotated')
    plotgraph(xh, yh, (-415, 415), (-415, 415), 'XHit, mm', 'YHit, mm', True, x_i, y_i, filename='XY')
  else:
    plotgraph(xh_r, yh_r, (-415, 415), (-415, 415), 'XHit_rotated, mm', 'YHit_rotated, mm', filename='XY_rotated')
    plotgraph(xh, yh, (-415, 415), (-415, 415), 'XHit, mm', 'YHit, mm', filename='XY')
    plotgraph(None, None, (-415, 415), (-415, 415), 'XIntersection, mm', 'YIntersection, mm', True, x_i, y_i, filename='IntersectionsXY')

  plotgraph(slices, counts, xlim=(0, 100), xlabel='Slices, mm', ylabel='Counts per slice', filename='SlidingWindow')
  plotgraph(x=phi_c_r, y=theta_c_r, xlim=(-np.pi*1.05, np.pi*1.05), ylim=(0., np.arccos(1/1.05)*2), xlabel=r'$\phi$ rotated, rad', ylabel=r'$\theta$ rotated, rad', filename='ThetaPhi_rotated')
  plotgraph(x=phi_c, y=theta_c, xlim=(-np.pi*1.05, np.pi*1.05), ylim=(0., np.arccos(1/1.05)*2), xlabel=r'$\phi$, rad', ylabel=r'$\theta$, rad', filename='ThetaPhi')

  plotgraph(x=phi_c, y=r_c, xlim=(0., np.arccos(1/1.05)*2), xlabel=r'$\phi_c$', ylabel='Distance to Intersection, mm', filename='R_cPhi_c')
  plotgraph(x=theta_c, y=r_c, xlim=(-np.pi*1.05, np.pi*1.05), xlabel=r'$\theta_c$', ylabel='Distance to Intersection, mm', filename='R_cTheta_c')
  plotgraph(x=th, y=theta_p, xlabel='THit, ns', ylabel=r'$\theta_p$', filename='Theta_p_t_c.png')
  plotgraph(x=bh, y=nh, xlabel=r'$\beta$', ylabel='NHits', filename='NHits_Beta')
  plotgraph(x=bh, y=th, xlabel=r'$\beta$', ylabel='THit, ns', filename='t_c_Beta')
  plotgraph(x=th, y=nh, xlabel='THit, ns', ylabel='NHits', filename='NHits_THit.png')

  plotgraph(x=phi_c_r, y=r_c_r,  xlabel='Phi, rad', ylabel='Rotated distance to intersection, mm', ylim=(0, 200), filename='RotatedR_cPhi_c')

  # plotgraph(x=true_beta, y=delta_beta, xlabel='True Beta', ylabel='Delta Beta', filename='TrueBeta_DeltaBeta')
  # plotgraph(x=true_beta, y=eps_beta, xlabel='True Beta', ylabel='Epsilon Beta', filename='TrueBeta_EpsilonaBeta')
  # plotgraph(x=calc_beta, y=delta_beta, xlabel='Calculated Beta', ylabel='Delta Beta', filename='CalceBeta_DeltaBeta')
  # plotgraph(x=calc_beta, y=eps_beta, xlabel='Calculated Beta', ylabel='Epsilon Beta', filename='CalcBeta_EpsilonaBeta')

  for n_sigms in range(1, 5):
    calc_r = edf[f'calculated_r_{n_sigms}_sigms'].to_numpy()
    delta_r = edf[f'delta_calc_true_r_{n_sigms}_sigms'].to_numpy()
    eps_r = edf[f'eps_calc_true_r_{n_sigms}_sigms'].to_numpy()
    calc_beta = edf[f'beta_from_calc_r_{n_sigms}_sigms'].to_numpy()
    delta_beta = edf[f'delta_beta_{n_sigms}_sigms'].to_numpy()
    eps_beta = edf[f'eps_beta_{n_sigms}_sigms'].to_numpy()

    plotgraph(x=true_r, y=delta_r, xlabel='True R', ylabel='Delta R', filename=f'TrueR_DeltaR_{n_sigms}_sigms')
    plotgraph(x=true_r, y=eps_r, xlabel='True R', ylabel='Epsilon R, %', filename=f'TrueR_EpsilonR_{n_sigms}_sigms')
    plotgraph(x=calc_r, y=delta_r, xlabel='Calculated R', ylabel='Delta R', filename=f'CalcR_DeltaR_{n_sigms}_sigms')
    plotgraph(x=calc_r, y=eps_r, xlabel='Calculated R', ylabel='Epsilon R, %', filename=f'CalcR_EpsilonR_{n_sigms}_sigms')

    plotgraph(x=true_beta, y=delta_beta, xlabel='True Beta', ylabel='Delta Beta', filename=f'TrueBeta_DeltaBeta_{n_sigms}_sigms')
    plotgraph(x=true_beta, y=eps_beta, xlabel='True Beta', ylabel='Epsilon Beta, %', filename=f'TrueBeta_EpsilonBeta_{n_sigms}_sigms')
    plotgraph(x=calc_beta, y=delta_beta, xlim=(0.95, 1), xlabel='Calculated Beta', ylabel='Delta Beta', filename=f'CalceBeta_DeltaBeta_{n_sigms}_sigms')
    plotgraph(x=calc_beta, y=eps_beta, xlim=(0.95, 1), xlabel='Calculated Beta', ylabel='Epsilon Beta, %', filename=f'CalcBeta_EpsilonBeta_{n_sigms}_sigms')

  #
  # figxy = plt.figure(figsize=(6,6))
  # plt.plot(xh[sh], yh[sh], 'r.', label='Signal')  # сигнальные срабатывания
  # if not sh.all():
  #   plt.plot(xh[~sh], yh[~sh], 'k.', label='Noise')  # шумовые срабатывания
  #   plt.legend()
  # if 'r_c' in edf.columns and 'x_i' in edf.columns and events == 1:
  #   plt.plot(x_i, y_i, 'g.', label='Intersection')
  #   plt.legend()
  # #plt.plot([-400, 400], [0, 0], 'k-')
  # #plt.plot([0, 0], [-400, 400], 'k-')
  # plt.xlim(-415, 415)
  # plt.ylim(-415, 415)
  # plt.xlabel('Xhit, mm', fontsize=14)
  # plt.ylabel('Yhit, mm', fontsize=14)
  # plt.tight_layout()
  # if dir_to_save != '':
  #   figxy.savefig(os.path.join(picsdir, dir_to_save, 'XY.png'))
  # plt.close(figxy)

  #plotgraph(None, None, (-415, 415), (-415, 415), 'XIntersection, mm', 'YIntersection, mm', True, x_i, y_i, 'IntersectionsXY')

  # if 'r_c' in edf.columns and 'x_i' in edf.columns:
  #   figintersections = plt.figure(figsize=(6,6))
  #   plt.plot(x_i, y_i, 'g.', label='Intersection')
  #   plt.legend()
  #   plt.xlim(-400, 400)
  #   plt.ylim(-400, 400)
  #   plt.xlabel('Xintersection, mm', fontsize=14)
  #   plt.ylabel('Yintersection, mm', fontsize=14)
  #   plt.tight_layout()
  #   if dir_to_save != '':
  #     figintersections.savefig(os.path.join(picsdir, dir_to_save, 'IntersectionsXY.png'))
  #   plt.close(figintersections)



  # figtime = plt.figure(figsize=(6,4))
  # if not sh.all():
  #   _, bins, _ = plt.hist([th[sh], th[~sh]], bins=100, color=['r', 'k'], histtype='stepfilled', label=['Signal', 'Noise'])
  #   plt.legend()
  # else:
  #   _, bins, _ = plt.hist(th[sh], bins=100, range=trange, color='r', histtype='stepfilled')
  # dt = bins[1] - bins[0]
  # plt.xlabel('Thit, ns', fontsize=14)
  # plt.ylabel(f'Hits/({dt:.3f} ns)', fontsize=14)
  # if (isinstance(trange, tuple) or isinstance(trange, list)) and len(trange) == 2:
  #   plt.xlim(trange)
  # #plt.semilogy()
  # plt.tight_layout()
  # if dir_to_save != '':
  #   figtime.savefig(os.path.join(picsdir, dir_to_save, 'T.png'))
  # plt.close(figtime)



  # if 't_c_orig' in edf.columns:
  #   figtime_orig = plt.figure(figsize=(6,4))
  #   plt.hist([toh[sh], toh[~sh]], bins=100, range=trange, color=['r', 'k'], histtype='stepfilled', label=['Signal', 'Noise'])
  #   plt.xlabel('Time of photon origin, ns', fontsize=14)
  #   plt.ylabel(f'Hits/({dt:.3f} ns)', fontsize=14)
  #   if not sh.all():
  #     plt.legend()
  #   #plt.semilogy()
  #   plt.tight_layout()
  #   if dir_to_save != '':
  #     figtime_orig.savefig(os.path.join(picsdir, dir_to_save, 'T_c_orig.png'))
  #   plt.close(figtime_orig)




  if 'theta_c' in edf.columns and 'phi_c' in edf.columns:
    # figangles = None
    # figangles = plt.figure(figsize=(6,4))
    # plt.plot(phi_c[sh], theta_c[sh], 'r.', label='Signal')
    # if not sh.all():
    #   plt.plot(phi_c[~sh], theta_c[~sh], 'k.', label='Noise')
    #   plt.legend()
    # #plt.xlim(-np.pi*1.05, np.pi*1.05)
    # #plt.ylim(0., np.arccos(1/1.05)*2)
    # plt.xlabel(r'$\phi_c$', fontsize=14)
    # plt.ylabel(r'$\theta_c$', fontsize=14)
    # plt.tight_layout()
    # if dir_to_save != '':
    #   figangles.savefig(os.path.join(picsdir, dir_to_save, 'Theta_cPhi_c.png'))
    # plt.close(figangles)

    figsmile = None
    if 'r_c' in edf.columns and 'x_i' in edf.columns:
      figsmile = plt.figure(figsize=(6,4))
      angle = np.empty(len(xh))
      for i in range(len(xh)):
        if xh[i] > x_i[i]:
          angle[i] = np.arcsin((yh[i] - y_i[i]) / r_c[i])
        else:
          if yh[i] > y_i[i]:
            angle[i] = np.pi -  np.arcsin((yh[i] - y_i[i]) / r_c[i])
          else:
            angle[i] = -np.pi / 2 + np.arcsin((yh[i] - y_i[i]) / r_c[i])
      plt.plot(angle[sh], r_c[sh], 'r.', label='Signal')
      if not sh.all():
        plt.plot(angle[~sh], r_c[~sh], 'k.', label='Noise')
        plt.legend()
      #plt.xlim(-np.pi*1.05, np.pi*1.05)
      #plt.ylim(0., np.arccos(1/1.05)*2)
      plt.xlabel(r'$\phi, rad$', fontsize=14)
      plt.ylabel('Distance to Intersection, mm', fontsize=14)
      plt.tight_layout()
      if dir_to_save != '':
        figsmile.savefig(os.path.join(picsdir, dir_to_save, 'R_cPhi.png'))
      plt.close(figsmile)


    if 'r_c' in edf.columns:
    #   figrphi = None
    #   figrphi = plt.figure(figsize=(6,4))
    #   plt.plot(phi_c[sh], r_c[sh], 'r.', label='Signal')
    #   if not sh.all():
    #     plt.plot(phi_c[~sh], r_c[~sh], 'k.', label='Noise')
    #     plt.legend()
    #   plt.xlim(0., np.arccos(1/1.05)*2)
    #   plt.ylabel('Distance to Intersection, mm', fontsize=14)
    #   plt.xlabel(r'$\phi_c$', fontsize=14)
    #   plt.tight_layout()
    #   if dir_to_save != '':
    #     figrphi.savefig(os.path.join(picsdir, dir_to_save, 'R_cPhi_c.png'))
    #   plt.close(figrphi)

      # figrtheta = None
      # figrtheta = plt.figure(figsize=(6,4))
      # plt.plot(theta_c[sh], r_c[sh], 'r.', label='Signal')
      # if not sh.all():
      #   plt.plot(theta_c[~sh], r_c[~sh], 'k.', label='Noise')
      #   plt.legend()
      # plt.xlim(-np.pi*1.05, np.pi*1.05)
      # plt.ylabel('Distance to Intersection, mm', fontsize=14)
      # plt.xlabel(r'$\theta_c$', fontsize=14)
      # plt.tight_layout()
      # if dir_to_save != '':
      #   figrtheta.savefig(os.path.join(picsdir, dir_to_save, 'R_cTheta_c.png'))
      # plt.close(figrtheta)


      # figthetat = plt.figure(figsize=(6,6))
      # plt.plot(t_c[sh], theta_p[sh], 'r.', label='Signal')  # сигнальные срабатывания
      # if not sh.all():
      #   plt.plot(t_c[~sh], theta_p[~sh], 'k.', label='Noise')  # шумовые срабатывания
      #   plt.legend()
      # plt.xlabel('t')
      # plt.ylabel('theta')
      # if dir_to_save != '':
      #   figthetat.savefig(os.path.join(picsdir, dir_to_save, 'Theta_p_t_c.png'))
      # plt.close(figthetat)
      fig3Drphitheta = None
      fig3Drphitheta = plt.figure(figsize=(12,8))
      ax = plt.axes(projection='3d')
      ax.scatter(phi_c[sh], theta_c[sh], r_c[sh], color='r', label='Signal')
      if not sh.all():
        ax.scatter(phi_c[~sh], theta_c[~sh], r_c[~sh], color='k', label='Noise')
        ax.legend()
      ax.set_xlabel(r'$\phi_c$', fontsize=14)
      ax.set_ylabel(r'$\theta_c$', fontsize=14)
      ax.set_zlabel('Distance to Intersection, mm', fontsize=14)
      #ax.tight_layout()
      if dir_to_save != '':
        fig3Drphitheta.savefig(os.path.join(picsdir, dir_to_save, '3DR_cPhi_cTheta_c.png'))
      plt.close(fig3Drphitheta)

  # fighitbeta = plt.figure(figsize=(6, 4))
  # plt.plot(bh[sh], nh[sh], 'r.', label='Signal')
  # if not sh.all():
  #     plt.plot(bh[~sh], nh[~sh], 'k.', label='Noise')
  #     plt.legend()
  # plt.ylabel('NHits', fontsize=14)
  # plt.xlabel(r'$\beta$', fontsize=14)
  # if dir_to_save != '':
  #   fighitbeta.savefig(os.path.join(picsdir, dir_to_save, 'NHits_Beta.png'))
  # plt.close(fighitbeta)

  # figtbeta = plt.figure(figsize=(6, 4))
  # plt.plot(bh[sh], t_c[sh], 'r.', label='Signal')
  # if not sh.all():
  #     plt.plot(bh[~sh], t_c[~sh], 'k.', label='Noise')
  #     plt.legend()
  # plt.ylabel('t_c, ns', fontsize=14)
  # plt.xlabel(r'$\beta$', fontsize=14)
  # if dir_to_save != '':
  #   figtbeta.savefig(os.path.join(picsdir, dir_to_save, 't_c_Beta.png'))
  # plt.close(figtbeta)

  # fighitt = plt.figure(figsize=(6, 4))
  # plt.plot(th[sh], nh[sh], 'r.', label='Signal')
  # if not sh.all():
  #     plt.plot(th[~sh], nh[~sh], 'k.', label='Noise')
  #     plt.legend()
  # plt.ylabel('NHits', fontsize=14)
  # plt.xlabel('THit, ns', fontsize=14)
  # if dir_to_save != '':
  #   fighitt.savefig(os.path.join(picsdir, dir_to_save, 'NHits_THit.png'))
  # plt.close(fighitt)


  if extended_mode:
    plot_extra_graph(xh, 'XHit', 'mm')
    plot_extra_graph(yh, 'YHit', 'mm')
    plot_extra_graph(zh, 'ZHit', 'mm')
    plot_extra_graph(edf['theta_c'], r'$\theta_c$', 'rad', pic_name='Theta')
    plot_extra_graph(edf['phi_c'], r'$phi_c$', 'rad', pic_name='Phi')
    plot_extra_graph(edf['r_c'], 'Distance to Intersection', 'mm')
    plot_extra_graph(edf['rotated_r_c'], 'Rotated distance to Intersection', 'mm')
    plot_extra_graph(toh, 'Time of photon origin', 'ns', pic_name='T_c_orig')
    plot_extra_graph(th, 'THit', 'ns', pic_name='T')
    for col in edf:
      fig = plot_extra_graph(edf[col], col, 'UNIT', raw=True)
      plt.close(fig)

  plt.show()
  #return figxy0, figxy, figtime, figangles, figx, figy, figz

In [ ]:
plothits(edf, 1000, 5, dir_to_save='10k_no_noise')

In [ ]:
ы

In [ ]:
#plothits(edf_with_noise, 1, 5)

In [ ]:
#plothits(edf_with_noise, 1, 1444)

In [ ]:
edf.loc[4363]

In [ ]:
plothits(edf, 1, 4363,dir_to_save='event_4363_0_noise')

In [ ]:
ы

In [ ]:
plothits(edf, 1, 1444 ,dir_to_save='event_1444_0_noise')

In [ ]:
plothits(edf, 1, 24 ,dir_to_save='event_24_0_noise')

In [ ]:
plothits(edf, 1, 5 ,dir_to_save='event_5_0_noise')

In [ ]:
ы

In [ ]:
edf

In [ ]:
for _, entry in edf.groupby(level=0):
  print(entry.shape[0])
  break

In [ ]:
ы

In [ ]:
t_edf.theta_p

In [ ]:
# fig, axs = plt.subplots(1, 2, figsize=(18, 9), sharey=True)
# fig.tight_layout(pad=-1.5)
plt.hist2d(edf.beta, edf.theta_p, bins=100)
# axs[0].set_xlim((-215, 215))
# axs[0].set_ylim((-215, 215))
plt.xlabel(r'$\beta$')
plt.ylabel(r'$\theta$, rad')
plt.show()

In [ ]:
# plt.hist(edf)

In [ ]:
thetas = []
betas = []
for _, entry in edf.groupby(level=0):
  for theta in entry.theta_p:
    thetas.append(np.cos(theta))
    break
  for beta in entry.beta:
    betas.append(beta)
    break
  # print(entry.theta_p)
  # break
thetas = np.array(thetas)
betas = np.array(betas)

In [ ]:
plt.hist(thetas, bins='auto')
plt.show()

In [ ]:
# t_edf = edf.loc[0:1000]
# t_edf = t_edf[t_edf['beta_group'] == 5]

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(9, 22.5), sharey=True)
fig.tight_layout(pad=-1.5)
for i in range(5):
  t_edf = edf.copy()
  t_edf = t_edf[t_edf['beta_group'] == i + 1]

  phi = np.arctan2(t_edf.x_c - t_edf.x_i, t_edf.y_c - t_edf.y_i)
  rotated_phi = np.arctan2(t_edf.rotated_x - t_edf.rotated_x_i, t_edf.rotated_y - t_edf.rotated_y_i)
  # axs[i, 0].hist2d(t_edf.x_c - t_edf.x_i, t_edf.y_c - t_edf.y_i, bins=100)
  axs[i, 0].hist2d((t_edf.x_c - t_edf.x_i) * np.cos(-t_edf.phi_p) - (t_edf.y_c - t_edf.y_i) * np.sin(-t_edf.phi_p),
                   (t_edf.x_c - t_edf.x_i) * np.sin(-t_edf.phi_p) + (t_edf.y_c - t_edf.y_i) * np.cos(-t_edf.phi_p), bins=100)

  axs[i, 0].set_xlim((-115, 115))
  axs[i, 0].set_ylim((-115, 115))
  axs[i, 1].hist2d((t_edf.rotated_x - t_edf.rotated_x_i) * np.cos(-t_edf.phi_p) - (t_edf.rotated_y - t_edf.rotated_y_i) * np.sin(-t_edf.phi_p),
                   (t_edf.rotated_x - t_edf.rotated_x_i) * np.sin(-t_edf.phi_p) + (t_edf.rotated_y - t_edf.rotated_y_i) * np.cos(-t_edf.phi_p), bins=100)
  axs[i, 1].set_xlim((-115, 115))
  axs[i, 1].set_ylim((-115, 115))
plt.show()

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(9, 22.5), sharey=True)
fig.tight_layout(pad=-1.5)
for i in range(5):
  t_edf = edf.loc[0:10000]
  t_edf = t_edf[t_edf['beta_group'] == i + 1]
  # t_edf = t_edf[t_edf.theta_p <= 0.55]
  # t_edf = t_edf[t_edf.theta_p >= 0.25]
  phi = np.arctan2(t_edf.x_c - t_edf.x_i, t_edf.y_c - t_edf.y_i)
  rotated_phi = np.arctan2(t_edf.rotated_x - t_edf.rotated_x_i, t_edf.rotated_y - t_edf.rotated_y_i)
  axs[i, 0].hist2d(t_edf.phi_c, t_edf.r_c, bins=100)
  # axs[i, 0].set_xlim((-115, 115))
  axs[i, 0].set_ylim((0, 215))
  axs[i, 1].hist2d(t_edf.phi_c, t_edf.rotated_r_c, bins=100)
  # axs[i, 1].set_xlim((-115, 115))
  axs[i, 1].set_ylim((0, 215))
plt.show()

In [ ]:
t_edf = edf.loc[0:10000]
t_edf = t_edf[t_edf.theta_p <= 0.55]
t_edf = t_edf[t_edf.theta_p >= 0.25]
plt.hist2d(t_edf.calculated_r_4_sigms, t_edf.beta,bins=80, range=((0, 70), (min(t_edf.beta), max(t_edf.beta))))
plt.show()

In [ ]:
plt.hist(np.cos(bdf.theta_p))

In [ ]:
edf_d.signal_counts

In [ ]:
ы

# With DCR

In [64]:
event = 1444
#momentum = edf.at[(event, 0), 'momentum']
#theta_p = edf.at[(event, 0), 'theta_p']*180/np.pi
avg_sigmas=(4, 5)
avg_t_sigmas=(4, 5)
num_of_groups = 10
r_width = float(idf.pixel_size)
t_width = 0.25
t_step = 0.25
r_factor = 2 # not to change
t_factor = int(t_width / t_step)
weighed = False
deg_lim = False
param_fit = False
for t_width in [0.25]: # [0.25, 0.5, 0.75, 1]
  for t_step in [0.25]: # [0.25, 0.5]
    for r_width in [2 * float(idf.pixel_size)]: # [float(idf.pixel_size), 2 * float(idf.pixel_size)]
      for weighed in [True]: # [False, True]
        print(f'Generating: {t_width}_{t_step}_{r_width}_{weighed}\n')
        cal_arr_for_dcr = False
        beta_sigms = []
        beta_sigms_yerr = []
        beta_sigms_deglim = []
        beta_sigms_yerr_deglim = []
        for dcr in ['0']  : # ['0', '1e3', '1e4', '1e5', '1e6']
          timer_start = perf_counter()
          gen = genChunkFromRoot(filepath, 10000, noisefreqpersqmm=float(dcr), shiftSignalTimes=True)
          c_bdf_d = pd.DataFrame()
          for i in range(20):
            edf_d = next(gen)
            # edf_d = next(genChunkFromRoot(filepath, 10000, noisefreqpersqmm=float(dcr), shiftSignalTimes=True))

            bdf_d = pd.DataFrame()
            # signalLength(edf_d)
            recoAngles(edf_d, idf)
            edf_d = applySpaceCut(edf_d)
            planeRecalculation(edf_d, idf)
            planeRotation(edf_d)
            edf_d = applySecondSpaceCut(edf_d)
            primaryDirectionRecalculation(edf_d)

            recoAngles(edf_d, idf, rotation_mode=True)
            cal_arr_for_dcr = rSlidingWindow(edf_d, idf, bdf_d, avg_sigmas=avg_sigmas, avg_t_sigmas=avg_t_sigmas, cal_arr=cal_arr_for_dcr, num_of_groups=num_of_groups,
                                             step=r_width, t_window_width=t_width, r_width_factor=r_factor, t_width_factor=t_factor, weighed=weighed, deg_lim=deg_lim, param_fit=param_fit)

            if deg_lim:
              edf_d = edf_d[edf_d.theta_p <= 10. * np.pi / 180]
              edf_d = edf_d[edf_d.signal_counts >= 5]
              bdf_d = bdf_d[bdf_d.theta_p <= 10. * np.pi / 180]
              bdf_d = bdf_d[bdf_d.signal_counts >= 5]
            if i == 0:
              c_bdf_d = bdf_d
            else:
              c_bdf_d = pd.concat([c_bdf_d, bdf_d], ignore_index=True)
          plot_groupped_distributions(c_bdf_d, avg_sigmas, avg_t_sigmas, r_width, t_width, r_factor, t_factor, weighed, background=dcr, deg_lim=False, num_of_groups=num_of_groups, to_save=True)
          bg = betaGroupsRMS90(c_bdf_d, avg_sigmas=avg_sigmas, avg_t_sigmas=avg_t_sigmas, n=num_of_groups)
          beta_sigms.append(bg[0])
          beta_sigms_yerr.append(bg[2])

          c_bdf_d = c_bdf_d[c_bdf_d.theta_p <= 10. * np.pi / 180]
          c_bdf_d = c_bdf_d[c_bdf_d.signal_counts >= 5]

          plot_groupped_distributions(c_bdf_d, avg_sigmas, avg_t_sigmas, r_width, t_width, r_factor, t_factor, weighed, background=dcr, deg_lim=True, num_of_groups=num_of_groups, to_save=True)
          bg_deglim = betaGroupsRMS90(c_bdf_d, avg_sigmas=avg_sigmas, avg_t_sigmas=avg_t_sigmas, n=num_of_groups)
          beta_sigms_deglim.append(bg_deglim[0])
          beta_sigms_yerr_deglim.append(bg_deglim[2])
          print(perf_counter() - timer_start)

          # plothits(edf_d, 1, event, dir_to_save=f'event_{event}_{dcr}_noise')

          #figxy.savefig(os.path.join(picsdir, f'labeled_ring_pi_p{momentum:.0f}mev_theta{theta_p:.0f}deg_dcr{dcr:.3g}.png'))
          #figtime.savefig(os.path.join(picsdir, f'labeled_time_pi_p{momentum:.0f}mev_theta{theta_p:.0f}deg_dcr{dcr:.3g}.png'))
        beta_sigms = np.array(beta_sigms)
        beta_sigms_yerr = np.array(beta_sigms_yerr)
        beta_sigms_deglim = np.array(beta_sigms_deglim)
        beta_sigms_yerr_deglim = np.array(beta_sigms_yerr_deglim)
        plot_final_graph(beta_sigms, beta_sigms_yerr, avg_sigmas, avg_t_sigmas, r_width, t_width, r_factor, t_factor, weighed, deg_lim=False, num_of_groups=num_of_groups)

        plot_final_graph(beta_sigms_deglim, beta_sigms_yerr_deglim, avg_sigmas, avg_t_sigmas, r_width, t_width, r_factor, t_factor, weighed, deg_lim=True, num_of_groups=num_of_groups)

Generating: 0.25_0.25_6.32_True

    Generate noise with DCR per mm^2 0.0, mean number of hits per event: 0.00.
    Generate noise with DCR per mm^2 0.0, mean number of hits per event: 0.00.
    Generate noise with DCR per mm^2 0.0, mean number of hits per event: 0.00.
    Generate noise with DCR per mm^2 0.0, mean number of hits per event: 0.00.
    Generate noise with DCR per mm^2 0.0, mean number of hits per event: 0.00.
    Generate noise with DCR per mm^2 0.0, mean number of hits per event: 0.00.
    Generate noise with DCR per mm^2 0.0, mean number of hits per event: 0.00.
    Generate noise with DCR per mm^2 0.0, mean number of hits per event: 0.00.
    Generate noise with DCR per mm^2 0.0, mean number of hits per event: 0.00.
    Generate noise with DCR per mm^2 0.0, mean number of hits per event: 0.00.
    Generate noise with DCR per mm^2 0.0, mean number of hits per event: 0.00.
    Generate noise with DCR per mm^2 0.0, mean number of hits per event: 0.00.
    Generate noise 

In [62]:
cal_arr_for_dcr

array([[[[ 9.62677888e-01,  9.43335679e+00,  3.19068569e+02],
         [ 9.60224080e-01,  2.86307332e+00,  3.35198850e+02],
         [ 9.63044237e-01,  1.14927966e+01,  2.90680811e+02],
         [ 9.58416953e-01,  2.59870292e-01,  3.23216407e+02],
         [ 9.56117215e-01, -5.62534814e+00,  3.36518761e+02],
         [ 9.47356731e-01, -2.40784608e+01,  3.78789659e+02],
         [ 9.55046524e-01, -8.06193363e+00,  3.25206517e+02],
         [ 9.55635625e-01, -6.74519733e+00,  3.10747679e+02],
         [ 9.51678885e-01, -1.33695578e+01,  3.17885734e+02],
         [ 9.46982256e-01, -2.14313133e+01,  3.30003956e+02]]]])

In [ ]:
edf_d

In [ ]:
signalLength(edf_d)
edf_to_bdf(edf_d.signal_length, bdf_d)

In [ ]:
edf_cut_05 = edf_d[(edf_d.theta_p <= 0.087) & (edf_d.beta <= 0.996) & (edf_d.rotated_r_c <= 100)]
edf_cut_40 = edf_d[(edf_d.theta_p >= 0.698) & (edf_d.beta <= 0.996) & (edf_d.rotated_r_c <= 100)]

bdf_cut_05 = bdf_d[(bdf_d.theta_p <= 0.087) & (bdf_d.beta <= 0.996) ]
bdf_cut_40 = bdf_d[(bdf_d.theta_p >= 0.698) & (bdf_d.beta <= 0.996)]

In [ ]:
arr = []
for i in range(9):
  edf_cut = edf_d[(edf_d.theta_p <= 0.087 * (i + 1)) & (edf_d.theta_p >= 0.087 * i) & (edf_d.rotated_r_c <= 55 + 5 * i)]

  arr.append(np.std(edf_cut.rotated_r_c))

In [ ]:
plt.plot([i for i in range(9)], arr)
plt.show()

In [ ]:
def gaussian(x, mean, sigma):
  return norm.pdf(x, loc=mean, scale=sigma)
bin_heights, bin_borders = np.histogram(edf_cut.rotated_r_c, bins='auto', normed=True)
bin_width = np.diff(bin_borders)
bin_centers = bin_borders[:-1] + bin_width / 2
p0 = norm.fit(edf_cut.rotated_r_c)
try:
  gauss_param, cov = curve_fit(gaussian, bin_centers, bin_heights, maxfev=50000, p0=p0, method='trf')
except ValueError:
  gauss_param, cov = curve_fit(gaussian, bin_centers, bin_heights, maxfev=50000, method='trf')
  cov = np.sqrt(np.diag(cov))


In [ ]:
gauss_param

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].hist(bdf_cut_05.signal_length, bins='auto')
axs[1].hist(bdf_cut_40.signal_length, bins='auto')
plt.show()

In [ ]:
print(np.std(bdf_cut_05.signal_length))
print(np.std(bdf_cut_40.signal_length))

In [ ]:
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(111, projection='3d')
x = np.linspace(0., 80., num=100)
y = np.linspace(theta_p_min, theta_p_max, 100)
X, Y = np.meshgrid(x, y)
zs = np.array(d3pol2((np.ravel(X), np.ravel(Y)), *cal_arr_for_dcr[0][0].flatten()))
zs[np.where(zs>1)[0]] = 1
Z = zs.reshape(X.shape)

z2 = np.array([[1.] * 100] * 100)
Z2 = z2.reshape(X.shape)

# ax.plot_surface(X, Y, Z2)
ax.plot_surface(X, Y, Z)
# ax.set_zlim((0.955, 1))
ax.set_xlabel('R, mm')
ax.set_ylabel(r'$cos\theta_p$')
ax.set_zlabel(r'$\beta$')

plt.show()

In [ ]:
zs[np.where(zs>1)[0]] = 1

In [ ]:
cal_arr = np.array(cal_arr)

In [ ]:
cal_arr_for_dcr[0][0].flatten()

In [ ]:
cal_arr_for_dcr[0][0]

In [ ]:
cal_arr[3, :, 0]

In [ ]:
theta_dif = (theta_intervals[1:] + theta_intervals[:-1]) / 2

In [ ]:
theta_dif

In [ ]:
c1, cerrs = curve_fit(pol2, theta_dif, cal_arr[3,:,2])

In [ ]:
plt.scatter(theta_dif, cal_arr[3,:,2])
plt.plot(theta_dif, pol2(theta_dif, *c1))
plt.show()

In [ ]:
edf_d.signal_length

In [ ]:
sig_lengths = [sub.iloc[0] for _, sub in edf_d.signal_length.groupby(level=0)]

In [ ]:
plt.hist(sig_lengths, bins='auto')
plt.xlabel('Signal Lenght, ns')
plt.show()

In [ ]:
c_bdf_d

In [ ]:
bdf_d

In [ ]:
if np.isnan(beta_sigms[0][0][0][0]):
  print(1)
print(type(beta_sigms[0][0][0][0]))
print(type(np.nan))

In [ ]:
beta_sigms

In [ ]:
signalLength(edf_d)

In [ ]:
plt.scatter(ev.slice_group, ev.slice_counts_4_sigms)
plt.xlim((0, 100))
plt.show()

In [ ]:
edf_d

In [ ]:
ev = edf_d.loc[1444]

In [ ]:
ev

In [ ]:
ev = edf_d.loc[190005]

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(ev[ev.signal == True].x_c, ev[ev.signal == True].y_c, c='r', s=1)
plt.scatter(ev[ev.signal == False].x_c, ev[ev.signal == False].y_c, s=1)
plt.xlim((-415, 415))
plt.ylim((-415, 415))
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(ev[ev.signal == True].rotated_x, ev[ev.signal == True].rotated_y, c='r', s=1)
plt.scatter(ev[ev.signal == False].rotated_x, ev[ev.signal == False].rotated_y, s=1)
plt.xlim((-415, 415))
plt.ylim((-415, 415))
plt.show()

In [ ]:
plt.hist2d(ev.rotated_r_c, ev.t_c, bins=30)
plt.show()

In [ ]:
theta_p_max = max(bdf_d.cos_theta_p)
theta_p_min = min(bdf_d.cos_theta_p)
num_of_theta_intervals = 11
theta_intervals = np.linspace(theta_p_min, theta_p_max, num=num_of_theta_intervals)

In [ ]:
theta_intervals

In [ ]:
cut_bdf_d = bdf_d[bdf_d.cos_theta_p <= theta_intervals[5]]
cut_bdf_d = cut_bdf_d[cut_bdf_d.cos_theta_p >= theta_intervals[4]]

In [ ]:
cut_bdf_d

In [ ]:
# plt.figure(figsize=(7, 4))
plt.hist(cut_bdf_d.delta_beta_4_rsigms_4_tsigms, bins='auto')
plt.xlabel(r'$\beta_{residuals}$')
plt.title(r'Calibration for $cos\theta_p$ in ' + f'[{round(theta_intervals[9], 2)}, {round(theta_intervals[10], 2)}]\n'
                                                       f'Mean: {round(np.mean(cut_bdf_d.delta_beta_4_rsigms_4_tsigms), 4)}, Std: {round(np.std(cut_bdf_d.delta_beta_4_rsigms_4_tsigms), 3)}')
plt.xlim((-0.04, 0.08))
plt.show()

In [ ]:
cal_arr_copy

In [ ]:
cal_arr_copy[:,0] = [round(x, 3) for x in cal_arr_copy[:,0]]

In [ ]:
errs_copy[0]

In [ ]:
to_pres = np.array([np.array([f'{x} ' + r'±' + f' {y}' for x, y in zip(xs, ys)]) for xs, ys in zip(cal_arr_copy, errs_copy)])

In [ ]:
to_pres_2 = np.array([f'{round(theta_intervals[i], 2)} - {round(theta_intervals[i + 1], 2)}' for i in range(10)])

In [ ]:
to_pres_2

In [ ]:
to_pres

In [ ]:
import openpyxl
to_pr = pd.DataFrame(to_pres_2)

## save to xlsx file

filepath = 'test2.xlsx'

to_pr.to_excel(filepath, index=False)

In [ ]:
with open('test2.xlsx') as trg:
  pass

In [ ]:
min(edf_d.beta)

In [ ]:
max(edf_d.beta)

In [ ]:
fig, ax = plt.subplots()
h = ax.hist2d(ev.rotated_r_c, ev.t_c, bins=30)
# ax.xlabel('R, mm')
# ax.ylabel('T, ns')
fig.colorbar(h[3], ax=ax)
plt.show()

In [ ]:
t_edf_d = edf_d[edf_d['signal_counts'] >= 0]
# t_edf_d = t_edf_d[t_edf_d.theta_p <= 100.1]

In [ ]:
sorted_ind = []
t = -1
for index, _ in t_edf_d.sort_values('eps_beta_4_sigms').index:
  if index != t:
    t = index
    sorted_ind.append(t)
sorted_ind = np.array(sorted_ind)

In [ ]:
sorted_ind[-5:-1]

In [ ]:
print(max(t_edf_d['eps_beta_4_sigms']))
# MAX: 511, MIN: 988
print([i for i, j in t_edf_d.loc[t_edf_d['eps_beta_4_sigms'] == max(t_edf_d['eps_beta_4_sigms'])]['beta_group'].groupby(level=0)][0])

In [ ]:
print(min(t_edf_d['eps_beta_4_sigms']))
# MAX: 511, MIN: 988
print([i for i, j in t_edf_d.loc[t_edf_d['eps_beta_4_sigms'] == min(t_edf_d['eps_beta_4_sigms'])]['beta_group'].groupby(level=0)][0])

In [ ]:
# ev261.beta

In [ ]:
print(sorted_ind)

In [ ]:
ev261 = t_edf_d.loc[4906] # MAX: 511, MIN: 988
fig, axs = plt.subplots(1, 3, figsize=(21, 6))
# plt.figure(figsize=(6, 6))
axs[0].scatter(ev261['x_c'], ev261['y_c'], s=10, label='Original')
axs[0].scatter(ev261['rotated_x'], ev261['rotated_y'], s=10, label='Rotated')
axs[0].scatter(ev261['rotated_x_i'], ev261['rotated_y_i'])

angles = np.linspace(0, 2 * np.pi, num=ev261['rotated_x_i'].shape[0])
c_x = ev261['rotated_x_i'] + ev261.unfixed_calculated_r_4_sigms * np.cos(angles)
c_y = ev261['rotated_y_i'] + ev261.unfixed_calculated_r_4_sigms * np.sin(angles)

true_x = ev261['rotated_x_i'] + ev261.true_r_from_beta * np.cos(angles)
true_y = ev261['rotated_y_i'] + ev261.true_r_from_beta * np.sin(angles)

axs[0].plot(true_x, true_y, c='r', linewidth=1, linestyle='--', label='True Projected')
axs[0].plot(c_x, c_y, c='b', linewidth=1, linestyle='--', label='Measured')

axs[0].set_xlim((-415, 415))
axs[0].set_ylim((-415, 415))
axs[0].set_xlabel('X, mm')
axs[0].set_ylabel('Y, mm')
axs[0].legend()

# axs[1].scatter(ev261['phi_c'], ev261['r_c'], label='Original')
axs[1].scatter(ev261['phi_c'], ev261['rotated_r_c'], label='Rotated')
# axs[1].plot(np.linspace(-3, 3, 40), [list(ev261.corrected_true_r_from_beta)[0]] * 40, c='r', label='True Projected', linestyle='--')
axs[1].plot(np.linspace(-3, 3, 40), [list(ev261.unfixed_calculated_r_4_sigms)[0]] * 40, c='b', label='Measured', linestyle='--')
axs[1].set_ylim(10, 90)
axs[1].legend()
axs[1].set_xlabel(r'$\varphi$, rad')
axs[1].set_ylabel('R, mm')

axs[2].scatter(ev261['slice_group'], ev261['slice_counts_4_sigms'])
# axs[2].plot([list(ev261.corrected_true_r_from_beta)[0]] * 40, np.linspace(0, max(ev261.slice_counts), 40), c='r', linewidth=1, label='True Projected', linestyle='--')
axs[2].plot([list(ev261.unfixed_calculated_r_4_sigms)[0]] * 40, np.linspace(0, max(ev261.slice_counts_4_sigms), 40), c='b', linewidth=1, label='Measured', linestyle='--')
axs[2].set_xlim((10, 110))
axs[2].legend()
axs[2].set_xlabel('R, mm')
axs[2].set_ylabel(r'N / R,  $mm^{-1}$')

plt.show()

In [ ]:
plt.scatter(ev261.rotated_r_c, ev261.t_c)
plt.show()

In [ ]:
plt.hist2d(edf.rotated_r_c, edf.signal_length, bins=100, range=((0, 100), (0, 2)))
# plt.hist(edf.signal_length)
plt.show()

In [ ]:
plt.hist2d(t_bdf_d.unfixed_calculated_r_4_sigms, t_bdf_d.beta, bins=50)
plt.show()

In [ ]:
print(list(ev261.unfixed_calculated_r_4_sigms)[0])
# print(list(ev261.corrected_true_r_from_beta)[0])

In [ ]:
ev261

In [ ]:
edf_d

In [ ]:
bdf_d

In [ ]:
t_bdf_d = bdf_d[bdf_d['signal_counts'] >= 0]
# t_bdf_d = t_bdf_d[t_bdf_d['theta_p'] <= 0.7]
# t_bdf_d = t_bdf_d[(t_bdf_d.theta_p >= 0.45)]
# t_bdf_d = t_bdf_d[t_bdf_d.theta_p <=0.55]
t_bdf_d.replace([np.inf, -np.inf], np.nan, inplace=True)
t_bdf_d = t_bdf_d.dropna()
sigm = 4
t_sigm = 2
beta_meas_vs_beta_real = t_bdf_d[f'beta_from_calc_r_{sigm}_rsigms_{t_sigm}_tsigms'] / t_bdf_d.beta - 1

fig, axs = plt.subplots(1, 2, figsize=(16, 8))

axs[0].hist(beta_meas_vs_beta_real, bins='auto', alpha=1)
axs[0].set_xlabel(r'$\beta_{meas}$ / $\beta_{real}$ - 1')
axs[0].set_title(f'Window width = {sigm}$\sigma$\nMean = {round(np.mean(beta_meas_vs_beta_real), 3)}, Std = {round(np.std(beta_meas_vs_beta_real), 3)}')

axs[1].hist(beta_meas_vs_beta_real, bins='auto', alpha=1)
axs[1].set_xlabel(r'$\beta_{meas}$ / $\beta_{real}$ - 1')
axs[1].set_title(f'Window width = {sigm}$\sigma$\nMean = {round(np.mean(beta_meas_vs_beta_real), 3)}, Std = {round(np.std(beta_meas_vs_beta_real), 3)}')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 5), sharey='row')
fig.tight_layout(pad= -1.6)

axs[0].hist2d(t_bdf_d.beta, beta_meas_vs_beta_real, bins=100)
axs[1].hist2d(t_bdf_d.theta_p, beta_meas_vs_beta_real, bins=100)
axs[2].hist2d(t_bdf_d.signal_counts, beta_meas_vs_beta_real, bins=([i - 1 for i in range(90)], [min(beta_meas_vs_beta_real) + (max(beta_meas_vs_beta_real) - min(beta_meas_vs_beta_real)) / 90 * i for i in range(90)]))
axs[0].set_xlabel(r'$\beta$')
axs[1].set_xlabel(r'$\theta$')
axs[2].set_xlabel('NHits')
axs[0].set_ylabel(r'$\beta_{meas}$ / $\beta_{real}$ - 1')

axs[0].plot([0.955, 0.999], [0, 0], c='r', linewidth=1.5, alpha=0.4, linestyle='--')
axs[1].plot([0.01, 0.8], [0, 0], c='r', linewidth=1.5, alpha=0.4, linestyle='--')
axs[2].plot([1, 90], [0, 0], c='r', linewidth=1.5, alpha=0.4, linestyle='--')

#axs[0].set_ylim((-1, 2))
plt.show()

In [ ]:

plt.hist(t_bdf_d[f'eps_beta_{sigm}_rsigms_{t_sigm}_tsigms'], bins='auto')
plt.xlabel(r'$\epsilon_\beta$, %')
plt.title(f'Width = 4$\sigma$, DCR = $10^6$ cps\nMean = {round(np.mean(t_bdf_d[f"eps_beta_{sigm}_rsigms_{t_sigm}_tsigms"]), 3)}%, Width = {round(np.std(t_bdf_d[f"eps_beta_{sigm}_rsigms_{t_sigm}_tsigms"]), 3)}%')
plt.show()

In [ ]:
beta_sigms.shape # dcr, rsigm, tsigm, group

In [ ]:
print(beta_sigms[:,0])

In [ ]:
# beta_sigms_1 = np.array(beta_sigms[:,0])
# beta_sigms_2 = np.array(beta_sigms[:,1])
# beta_sigms_3 = np.array(beta_sigms[:,2])
# beta_sigms_4 = np.array(beta_sigms[:,3])
#
# beta_epss_1 = np.array(beta_epss[:,0])
# beta_epss_2 = np.array(beta_epss[:,1])
# beta_epss_3 = np.array(beta_epss[:,2])
# beta_epss_4 = np.array(beta_epss[:,3])

In [ ]:
# plt.figure(figsize=(16, 9))
# plt.plot(x, beta_sigms_1[0], label='1$\sigma$')
# plt.plot(x, beta_sigms_2[0], label='2$\sigma$')
# plt.plot(x, beta_sigms_3[0], label='3$\sigma$')
# plt.plot(x, beta_sigms_4[0], label='4$\sigma$')
# plt.title(r'Velocity resoultion for DCR = 0, $\theta < \pi/4$')
# plt.xlabel('Beta Group')
# plt.ylabel(r'$\Delta\beta$')
# plt.legend()
# plt.grid()
# plt.show()

In [ ]:
plt.hist(bdf_d.beta)
plt.show()

In [ ]:
plt.hist2d(bdf_d.signal_counts, bdf_d.beta, 60)
plt.show()

In [ ]:
# labels = ['0', '1e3', '1e4', '1e5', '1e6']
# labels = ['DCR = ' + i + ' cps' for i in labels]
# colors = ['c', 'y', 'g', 'r', 'm']
#
# y = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
# x = (y * (max(edf['beta']) - min(edf['beta'])) - max(edf['beta']) + 11 * min(edf['beta'])) / 10
#
# fig, axs = plt.subplots(np.ptp(avg_sigmas), np.ptp(avg_t_sigmas), figsize=(10 * np.ptp(avg_t_sigmas), 10 * np.ptp(avg_sigmas)))
# fig.suptitle('Method: N(r) / r')
#
# if np.ptp(avg_sigmas) > 1:
#   for i in range(np.ptp(avg_sigmas)):
#     for j in range(np.ptp(avg_t_sigmas)):
#       for k in range(beta_sigms_gauss.shape[0]):
#         axs[i, j].plot(x, beta_sigms_gauss[k, i, j], label=labels[k])
#       axs[i, j].legend(loc='upper right')
#       axs[i, j].set_xlabel('Beta Group')
#       axs[i, j].set_ylabel(r'$\Delta\beta$')
#       axs[i, j].set_title(f'Velocity resoultion for\nr window width = {avg_sigmas[0] + i}$\sigma$\nt window width = {avg_t_sigmas[0] + j}$\sigma$')
#       axs[i, j].grid()
# else:
#   for j in range(np.ptp(avg_t_sigmas)):
#     for k in range(beta_sigms_gauss.shape[0]):
#       axs[j].plot(x, beta_sigms_gauss[k, 0, j], label=labels[k], c=colors[k])
#       axs[j].errorbar(x, beta_sigms_gauss[k, 0, j], xerr=[np.diff(x)[0]/4 for _ in x], linestyle='', c=colors[k])
#       axs[j].errorbar(x, beta_sigms_gauss[k, 0, j], yerr=beta_sigms_yerr[k, 0, j], linestyle='', c=colors[k])
#     axs[j].legend(loc='upper right')
#     axs[j].set_xlabel('Beta Group')
#     axs[j].set_ylabel(r'$\Delta\beta$')
#     axs[j].set_title(f'Velocity resoultion for\nr window width = {avg_sigmas[0]}$\sigma$\nt window width = {avg_t_sigmas[0] + j}$\sigma$')
#     # axs[j].set_ylim((0, .01))
#     axs[j].grid()
# plt.show()

In [ ]:
plot_final_graph(beta_sigms, beta_sigms_yerr, avg_sigmas, avg_t_sigmas, float(idf.pixel_size), 2, 2, 8)

In [ ]:
labels = ['0', '1e3', '1e4', '1e5', '1e6']
labels = ['DCR = ' + i + ' cps' for i in labels]
y = np.array([1, 2, 3, 4, 5])
x = (y * (max(edf['beta']) - min(edf['beta'])) - max(edf['beta']) + 6 * min(edf['beta'])) / 5

fig, axs = plt.subplots(2, 2, figsize=(20, 20))
fig.suptitle('Method: N(r) / r')

for i in range(beta_epss_1.shape[0]):
  axs[0,0].plot(x, beta_epss_1[i], label=f'{labels[i]}')
axs[0,0].legend(loc='lower right')
axs[0,0].set_xlabel('Beta Group')
axs[0,0].set_ylabel(r'$\epsilon\beta$, %')
axs[0,0].set_title('Velocity resoultion for window width = 1$\sigma$')
axs[0,0].grid()

for i in range(beta_epss_2.shape[0]):
    axs[0,1].plot(x, beta_epss_2[i], label=f'{labels[i]}')
axs[0,1].legend(loc='lower right')
axs[0,1].set_xlabel(r'$\beta$')
axs[0,1].set_ylabel(r'$\epsilon\beta$, %')
axs[0,1].set_title(r'Velocity resoultion for window width = 2$\sigma$, $\theta < \pi/4$')
axs[0,1].grid()

for i in range(beta_epss_3.shape[0]):
    axs[1,0].plot(x, beta_epss_3[i], label=f'{labels[i]}')
axs[1,0].legend(loc='lower right')
axs[1,0].set_xlabel(r'$\beta$')
axs[1,0].set_ylabel(r'$\epsilon\beta$, %')
axs[1,0].set_title(r'Velocity resoultion for window width = 3$\sigma$, $\theta < \pi/4$')
axs[1,0].grid()

for i in range(beta_epss_4.shape[0]):
    axs[1,1].plot(x, beta_epss_4[i], label=f'{labels[i]}')
axs[1,1].legend(loc='lower right')
axs[1,1].set_xlabel(r'$\beta$')
axs[1,1].set_ylabel(r'$\epsilon\beta$, %')
axs[1,1].set_title(r'Velocity resoultion for window width = 4$\sigma$, $\theta < \pi/4$')
axs[1,1].grid()

plt.show()

In [ ]:
labels = ['0', '1e3', '1e4', '1e5', '1e6']
labels = ['DCR = ' + i + ' cps' for i in labels]
y = np.array([1, 2, 3, 4, 5])
x = (y * (max(edf['true_r_from_beta']) - min(edf['true_r_from_beta'])) - max(edf['true_r_from_beta']) + 6 * min(edf['true_r_from_beta'])) / 5

fig, axs = plt.subplots(2, 2, figsize=(20, 20))
fig.suptitle('Method: N(r) / r')

for i in range(r_sigms_1.shape[0]):
  axs[0,0].plot(x, r_sigms_1[i], label=f'{labels[i]}')
axs[0,0].legend(loc='lower right')
axs[0,0].set_xlabel('r, mm')
axs[0,0].set_ylabel(r'$\Delta$ r, mm')
axs[0,0].set_title('r resoultion for window width = 1$\sigma$')
axs[0,0].grid()

for i in range(r_sigms_2.shape[0]):
    axs[0,1].plot(x, r_sigms_2[i], label=f'{labels[i]}')
axs[0,1].legend(loc='lower right')
axs[0,1].set_xlabel(r'r, mm')
axs[0,1].set_ylabel(r'$\Delta$ r, mm')
axs[0,1].set_title(r'r resoultion for window width = 2$\sigma$, $\theta < \pi/4$')
axs[0,1].grid()

for i in range(r_sigms_3.shape[0]):
    axs[1,0].plot(x, r_sigms_3[i], label=f'{labels[i]}')
axs[1,0].legend(loc='lower right')
axs[1,0].set_xlabel(r'r, mm')
axs[1,0].set_ylabel(r'$\Delta$ r, mm')
axs[1,0].set_title(r'r resoultion for window width = 3$\sigma$, $\theta < \pi/4$')
axs[1,0].grid()

for i in range(r_sigms_4.shape[0]):
    axs[1,1].plot(x, r_sigms_4[i], label=f'{labels[i]}')
axs[1,1].legend(loc='lower right')
axs[1,1].set_xlabel(r'r, mm')
axs[1,1].set_ylabel(r'$\Delta$ r, mm')
axs[1,1].set_title(r'r resoultion for window width = 4$\sigma$, $\theta < \pi/4$')
axs[1,1].grid()

plt.show()

In [ ]:
labels = ['0', '1e3', '1e4', '1e5', '1e6']
labels = ['DCR = ' + i + ' cps' for i in labels]
y = np.array([1, 2, 3, 4, 5])
x = (y * (max(edf['true_r_from_beta']) - min(edf['true_r_from_beta'])) - max(edf['true_r_from_beta']) + 6 * min(edf['true_r_from_beta'])) / 5

fig, axs = plt.subplots(2, 2, figsize=(20, 20))
fig.suptitle('Method: N(r) / r')

for i in range(r_epss_1.shape[0]):
  axs[0,0].plot(x, r_epss_1[i], label=f'{labels[i]}')
axs[0,0].legend(loc='lower right')
axs[0,0].set_xlabel('r, mm')
axs[0,0].set_ylabel(r'$\epsilon$ r, %')
axs[0,0].set_title('r resoultion for window width = 1$\sigma$')
axs[0,0].grid()

for i in range(r_epss_2.shape[0]):
    axs[0,1].plot(x, r_epss_2[i], label=f'{labels[i]}')
axs[0,1].legend(loc='lower right')
axs[0,1].set_xlabel(r'r, mm')
axs[0,1].set_ylabel(r'$\epsilon$ r, %')
axs[0,1].set_title(r'r resoultion for window width = 2$\sigma$, $\theta < \pi/4$')
axs[0,1].grid()

for i in range(r_epss_3.shape[0]):
    axs[1,0].plot(x, r_epss_3[i], label=f'{labels[i]}')
axs[1,0].legend(loc='lower right')
axs[1,0].set_xlabel(r'r, mm')
axs[1,0].set_ylabel(r'$\epsilon$ r, %')
axs[1,0].set_title(r'r resoultion for window width = 3$\sigma$, $\theta < \pi/4$')
axs[1,0].grid()

for i in range(r_epss_4.shape[0]):
    axs[1,1].plot(x, r_epss_4[i], label=f'{labels[i]}')
axs[1,1].legend(loc='lower right')
axs[1,1].set_xlabel(r'r, mm')
axs[1,1].set_ylabel(r'$\epsilon$ r, %')
axs[1,1].set_title(r'r resoultion for window width = 4$\sigma$, $\theta < \pi/4$')
axs[1,1].grid()

plt.show()

In [ ]:
ы

In [ ]:
ы

In [ ]:
#edf[(edf['theta_p'] > 44.9/180*np.pi) & (edf['momentum'] > 1500)]

In [ ]:
event = 1444
for dcr in ['0', '1e3', '1e4', '1e5', '1e6']:
  edf_d = next(genChunkFromRoot(filepath, event + 6, noisefreqpersqmm=float(dcr), shiftSignalTimes=False))
  bdf_d = pd.DataFrame()
  # signalLength(edf_d)
  recoAngles(edf_d, idf)
  edf_d = applySpaceCut(edf_d)
  planeRecalculation(edf_d)
  planeRotation(edf_d)
  edf_d = applySecondSpaceCut(edf_d)
  primaryDirectionRecalculation(edf_d)
  recoAngles(edf_d, idf, rotation_mode=True)
  rSlidingWindow(edf_d, idf, bdf_d, method='N/r')
  bg = betaGroups(edf_d, bdf_d)
  plothits(edf_d, 1, event, dir_to_save=f'event_{event}_{dcr}_noise')




In [ ]:
# recoAngles(edf_d, idf, rotation_mode=True)
# edf_d = applySecondSpaceCut(edf_d)
bdf_d = pd.DataFrame()
rSlidingWindow(edf_d, idf, bdf_d, optimal_width)
print(betaGroups(edf_d, bdf_d))
# plothits(edf_d, 1, event, dir_to_save=f'event_{event}_{dcr}_noise')

In [ ]:
edf_d.loc[1444]

In [ ]:
plothits(edf_d, 1000, 5, dir_to_save='10k_1e5_noise')

In [ ]:

for t in [0, 0.5, 1, 1.5, 2]:
  t_edf = edf_d[(edf_d['t_c'] >= t) & (edf_d['t_c'] <= t+2)]
  rSlidingWindow(t_edf, idf, optimal_width)
  # os.mkdir(f'pics\\event_{event}_{dcr}_THit{t}-{t+2}ns')
  # os.mkdir(f'pics\\event_{event}_{dcr}_THit{t}-{t+2}ns\\raw')
  plothits(t_edf, 1, event, dir_to_save=f'event_{event}_{dcr}_THit{t}-{t+2}ns')

In [ ]:
t_edf

In [ ]:
gen = genChunkFromRoot(filepath, 1000, noisefreqpersqmm=0)

In [ ]:
gen_edf_1 = next(gen)
gen_edf_2 = next(gen)

In [ ]:
gen_edf_2

In [ ]:
bdf_d = pd.DataFrame()

In [ ]:
conc_edf = bdf_d.combine_first(gen_edf_2)

In [ ]:
conc_edf

In [ ]:
b_1 = c_bdf_d.copy()
b_2 = c_bdf_d.copy()

In [ ]:
b = pd.concat([b_1, b_2], ignore_index=True)

In [ ]:
b